# Setup section

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!nvidia-smi

Tue Oct  8 16:39:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
# Download repositories

!sudo rm -dr GPUcomputing
!git clone https://github.com/S3gmentati0nFault/GPUcomputing.git
!mkdir logs

rm: cannot remove 'GPUcomputing': No such file or directory
Cloning into 'GPUcomputing'...
remote: Enumerating objects: 723, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 723 (delta 207), reused 306 (delta 154), pack-reused 347 (from 1)
Receiving objects: 100% (723/723), 2.81 MiB | 20.10 MiB/s, done.
Resolving deltas: 100% (364/364), done.


In [5]:
%cd GPUcomputing/utils/nvcc4jupyter-master/
!python3 -m build
%load_ext nvcc4jupyter
%cd /content/

/content/GPUcomputing/utils/nvcc4jupyter-master
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating nvcc4jupyter.egg-info
writing nvcc4jupyter.egg-info/PKG-INFO
writing dependency_links to nvcc4jupyter.egg-info/dependency_links.txt
writing top-level names to nvcc4jupyter.egg-info/top_level.txt
writing manifest file 'nvcc4jupyter.egg-info/SOURCES.txt'
reading manifest file 'nvcc4jupyter.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'nvcc4jupyter.egg-info/SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing nvcc4jupyter.egg-info/PKG-INFO
writing dependency_links to nvcc4jupyter.egg-info/dependency_links.txt
writing top-level names to nvcc4jupyter.egg-info/top_level.txt
reading manifest file 'nvcc4jupyter.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'nvcc4jupyter.egg-info/SOURCES.txt

In [6]:
# DeviceQuery dell'attuale device (su Colab!)

!nvcc -arch=sm_75 /content/GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery


CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  GPU arch name:                                 Turing
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (40) Multiprocessors, ( 64) CUDA Cores/MP:     2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory                65536

In [7]:
# Installation of the Valgrind utility

!sudo apt install valgrind

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  gdb libbabeltrace1 libc6-dbg libdebuginfod-common libdebuginfod1 libipt2
  libsource-highlight-common libsource-highlight4v5
Suggested packages:
  gdb-doc gdbserver valgrind-dbg valgrind-mpi kcachegrind alleyoop valkyrie
The following NEW packages will be installed:
  gdb libbabeltrace1 libc6-dbg libdebuginfod-common libdebuginfod1 libipt2
  libsource-highlight-common libsource-highlight4v5 valgrind
0 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 32.3 MB of archives.
After this operation, 111 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libdebuginfod-common all 0.186-1build1 [7,878 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libbabeltrace1 amd64 1.5.8-2build1 [160 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libdebuginfod1 amd6

# Configuration

Test interessanti:

SIZE 20000

MAX_WEIGHT 50000

FIXED_SEED 78651423

In [63]:
%%cuda_group_save --name "sharedMacros.h" --group "COMMON"

#ifndef SHARED_MACROS_H
#define SHARED_MACROS_H

#define DEBUGGING 0
#define SIZE 20000
#define MAX_WEIGHT 50000
#define TESTING 0
#define TEST_SIZE 3
#define FIXED_SEED 394867
#define LOGPATH "/content/logs/"
#define BLOCK_SIZE 1024

#endif

# CPU zone

## MST solution with Prim

In [44]:
%%cuda_group_save --name "mst.h" --group "CPU"

#ifndef MST_H
#define MST_H

// Header file di C++
#include <vector>

struct mst {
    std::vector<int> *stree;
    int totalWeight;

    mst() {
        stree = NULL;
        totalWeight = 0;
    }

    ~mst() {
        if (stree != NULL) {
            delete[] stree;
        }
        stree = NULL;
        totalWeight = 0;
    }
};

#endif

### Heap implementation

In [45]:
%%cuda_group_save --name "heap.h" --group "CPU"

// Header file di C++
#include <vector>

// Header file custom
#include "../COMMON/sharedMacros.h"

using namespace std;

#ifndef HEAP_H
#define HEAP_H

struct edge {
    edge () {
        source = UINT_MAX;
        destination = UINT_MAX;
        offset = UINT_MAX;
        weight = INT_MAX;
    }

    ~edge () {
        source = UINT_MAX;
        destination = UINT_MAX;
        offset = UINT_MAX;
        weight = INT_MAX;
    }

    edge (uint source, uint destination, uint offset, int weight) {
        this->source = source;
        this->destination = destination;
        this->offset = offset;
        this->weight = weight;
    }

    bool operator<(const edge& other) const {
        if (this->weight == other.weight) {
            return this->destination < other.destination;
        }
        return this->weight < other.weight;
    }

    bool operator>(const edge& other) const {
        if (this->weight == other.weight) {
            return this->destination > other.destination;
        }
        return this->weight > other.weight;
    }

    uint source, destination, offset;
    int weight;
};

class Heap {
    public:
        Heap () {
            size = 0;
            map = NULL;
        }

        Heap (uint size) {
            mapSize = size;
            map = new uint[mapSize];
            for (uint i = 0; i < mapSize; i++) {
                map[i] = UINT_MAX;
            }
            this->size = 0;
        }

        ~Heap () {
            delete[] map;
            map = NULL;
        }

        uint getLeftChild (uint index) {
            return 2 * index + 1;
        }

        uint getRightChild (uint index) {
            return 2 * index + 2;
        }

        uint getParent (uint index) {
            return (index - 1) / 2;
        }

        edge getKey (uint index) {
            return heap[index];
        }

        uint getPosition (uint key) {
            return map[key];
        }

        void insert (uint source, uint destination, uint offset, int weight) {
            // Insert the new edge
            edge newEdge = edge(source, destination, offset, weight);
            heap.push_back(newEdge);
            size++;

            map[destination] = size - 1;

            uint i = size - 1;
            while (i > 0 && heap[getParent(i)] > heap[i]) {
                swap(heap[i], heap[getParent(i)]);
                swap(map[heap[i].destination], map[heap[getParent(i)].destination]);
                i = getParent(i);
            }
        }

        void heapify (uint subtree) {
            uint left = getLeftChild(subtree);
            uint right = getRightChild(subtree);
            uint minimum = subtree;

            if (left < size && heap[left] < heap[minimum]) {
                minimum = left;
            }

            if (right < size && heap[right] < heap[minimum]) {
                minimum = right;
            }

            if (minimum != subtree) {
                swap(heap[subtree], heap[minimum]);
                swap(map[heap[subtree].destination], map[heap[minimum].destination]);

                heapify(minimum);
            }
        }

        void print() {
            for (uint i = 0; i < size; i++) {
                cout << "(" << heap[i].source << ", " << heap[i].destination << ")[" << heap[i].weight << "]" << endl;
            }
        }

        void printMap() {
            for (uint i = 0; i < mapSize; i++) {
                cout << i << ", " << map[i] << endl;
            }
        }

        edge peek() {
            return heap[0];
        }

        edge pop() {
            edge min;
            if (size == 0) {
                return min;
            }

            swap(heap[0], heap[size - 1]);
            swap(map[heap[0].destination], map[heap[size - 1].destination]);
            min = heap[size - 1];

            heap.erase(heap.end());
            map[min.destination] = UINT_MAX;
            size--;

            if (size == 0) {
                return min;
            }

            heapify(0);
            return min;
        }

        bool isEmpty() {
            if (size == 0) {
                return true;
            }
            return false;
        }

        void keyDecrease(uint i, uint source, int weight) {
            heap[i].weight = weight;
            heap[i].source = source;

            while (i > 0 && heap[getParent(i)] > heap[i]) {
                swap(heap[i], heap[getParent(i)]);
                swap(map[heap[i].destination], map[heap[getParent(i)].destination]);
                i = getParent(i);
            }
        }

    private:
        uint size, mapSize;
        uint *map;
        vector<edge> heap;
};

#endif

In [ ]:
%%cuda_group_save --name "heap.cu" --group "CPU"

#include <iostream>

#include "heap.h"

int main() {
    Heap *heap = new Heap();

    // void insert (uint source, uint destination, uint offset, int weight) {
    heap->insert(3, 0, 0, 10);
    heap->insert(3, 1, 0, 11);
    heap->insert(3, 2, 0, 12);
    heap->insert(3, 3, 0, 123);
    heap->insert(3, 4, 0, 14);
    heap->insert(3, 5, 0, 10);
    heap->insert(3, 6, 0, 16);
    heap->insert(3, 7, 0, 10);
    heap->insert(3, 8, 0, 18);
    heap->insert(3, 9, 0, 0);

    heap->print();

    printf("\n\n");

    heap->pop();

    heap->print();

    delete heap;

    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/heap.cu -o heap
!./heap

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/heap.cu -o heap
!valgrind ./heap

### Valgrind testing

In [ ]:
%%cuda_group_save --name "nonTimedMstCPUPrim.cu" --group "CPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "mst.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

mst *primMST (uint size, default_random_engine &eng, GraphStruct *str) {
    // Initialize the tree to the emptyset
    mst *tree = new mst;
    tree->stree = new vector<int>[size];
    tree->totalWeight = 0;

    // Initialize the vertex set U to a random vertex in the graph
    vector<node> U;
    vector<node> Remaining;
    uniform_real_distribution<> randR(0.0, 1.0);
    node randV;
    randV = (node)(randR(eng) * size);
    cout << "Source for the MST: " << randV << endl;
    U.push_back(randV);
    for (node i = 0; i < size; i++) {
        if (i != randV) {
            Remaining.push_back(i);
        }
    }

    // While the set U is different from the set V
    while (!Remaining.empty()) {

      /*
       * Find the edge with the lowest weight in the adjacency list of the
       * nodes currently in U
      */
      int minimum = INT_MAX;
      int sourceCandidate = -1;
      int destinationCandidate = -1;
      for (node i = 0; i < U.size(); i++) {
          for (uint j = 0; j < str->deg(U[i]); j++) {
              node neigh = str->getNeigh(U[i], j);
              uint occ = count(U.begin(), U.end(), neigh);
              if (occ == 0 && (str->getWeight(U[i], j) < minimum ||
                  (str->getWeight(U[i], j) == minimum && neigh < destinationCandidate))) {
                  minimum = str->getWeight(U[i], j);
                  sourceCandidate = U[i];
                  destinationCandidate = neigh;
              }
          }
      }

      if (sourceCandidate != -1) {
          if (DEBUGGING) {
              cout << "Source: " << sourceCandidate << " Destination: " << destinationCandidate << " Weight: " << minimum << endl;
          }

          // Add the newfound edge to the mst
          tree->stree[sourceCandidate].push_back(destinationCandidate);
          tree->totalWeight += minimum;

          // Add the destination vertex to the set U
          U.push_back(destinationCandidate);

          // Remove the element using erase function and iterators
          auto it = find(Remaining.begin(), Remaining.end(), destinationCandidate);

          // If element is found found, erase it
          if (it != Remaining.end()) {
              Remaining.erase(it);
          }
      }

      else {
          cout << "CPU-MST [Error]: The graph appears to be disconnected" << endl;
          return tree;
      }
    }

    return tree;
}

int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine fixedEng(FIXED_SEED);
    std::default_random_engine variableEng(rd());
    Graph *graph;
    char path[] = "/content/testing/primTest.txt";
    uint size = SIZE;
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 0;

    // Definition of testing variables
    uint repetitions;
    uint sizeArray[] = {50, 500, 1000};
    FILE *fptr;

    // Setup the testing variables
    if (TESTING) {
        repetitions = TEST_SIZE;
        fptr = fopen(path, "w");
        if (!fptr) {
            std::perror("File opening failed");
            return -1;
        }
    }
    else {
        repetitions = 1;
    }

    // Call to the Prim MST solver
    for (uint i = 0; i < repetitions; i++) {
        // Generation of the random graph
        if (TESTING) {
            size = sizeArray[i];
        }
        graph = new Graph(size, GPUEnabled);
        graph->randGraph(prob, true, maxWeight, fixedEng);

        // Test the printing procedure
        if (size < 15) {
            cout << "Printing the graph" << endl;
            graph->print(true);
        }

        cout << "Computing the MST solution for a graph of size " << size << endl;
        //cudaEventRecord(start);
        mst *tree = primMST(size, variableEng, graph->getStruct());
        if (size < 15) {
            cout << "MST solution" << endl;
            for (int i = 0; i < size; ++i) {
                cout << i << ": ";
                for (int j = 0; j < tree->stree[i].size(); ++j) {
                    cout << tree->stree[i][j] << " ";
                }
                cout << endl;
            }
        }
        cout << "Total weight: " << tree->totalWeight << endl;
        cout << "\n\n\n\n";
        if (TESTING) {
            fprintf(fptr, "%d,%d,\n", size, tree->totalWeight);
        }

        // Memory deallocation
        delete tree;
        delete graph;
        tree = NULL;
        graph = NULL;
    }

    if (TESTING) {
        fclose(fptr);
    }

    return 0;
}

In [ ]:
# Valgrind testing

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/nonTimedMstCPUPrim.cu -o valgrindCPU
!valgrind ./valgrindCPU

### Simple run

In [ ]:
%%cuda_group_save --name "mstCPUPrim.cu" --group "CPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "mst.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

mst *primMST (uint size, default_random_engine &eng, GraphStruct *str) {
    // Initialize the tree to the emptyset
    mst *tree = new mst();
    tree->stree = new vector<int>[size];
    tree->totalWeight = 0;

    // Initialize the vertex set U to a random vertex in the graph
    vector<node> U;
    vector<node> Remaining;
    uniform_real_distribution<> randR(0.0, 1.0);
    node randV;
    randV = (node)(randR(eng) * size);
    cout << "Source for the MST: " << randV << endl;
    U.push_back(randV);
    for (node i = 0; i < size; i++) {
        if (i != randV) {
            Remaining.push_back(i);
        }
    }

    // While the set U is different from the set V
    while (!Remaining.empty()) {

      /*
       * Find the edge with the lowest weight in the adjacency list of the
       * nodes currently in U
      */
      int minimum = INT_MAX;
      int sourceCandidate = -1;
      int destinationCandidate = -1;
      for (node i = 0; i < U.size(); i++) {
          for (uint j = 0; j < str->deg(U[i]); j++) {
              node neigh = str->getNeigh(U[i], j);
              uint occ = count(U.begin(), U.end(), neigh);
              if (occ == 0 && (str->getWeight(U[i], j) < minimum ||
                  (str->getWeight(U[i], j) == minimum && neigh < destinationCandidate))) {
                  minimum = str->getWeight(U[i], j);
                  sourceCandidate = U[i];
                  destinationCandidate = neigh;
              }
          }
      }

      if (sourceCandidate != -1) {
          if (DEBUGGING) {
              cout << "Source: " << sourceCandidate << " Destination: " << destinationCandidate << " Weight: " << minimum << endl;
          }

          // Add the newfound edge to the mst
          tree->stree[sourceCandidate].push_back(destinationCandidate);
          tree->totalWeight += minimum;

          // Add the destination vertex to the set U
          U.push_back(destinationCandidate);

          // Remove the element using erase function and iterators
          auto it = find(Remaining.begin(), Remaining.end(), destinationCandidate);

          // If element is found found, erase it
          if (it != Remaining.end()) {
              Remaining.erase(it);
          }
      }

      else {
          cout << "CPU-MST [Error]: The graph appears to be disconnected" << endl;
          return tree;
      }
    }

    return tree;
}

int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine fixedEng(FIXED_SEED);
    std::default_random_engine variableEng(rd());
    Graph *graph;
    char path[] = "/content/testing/cpuTest.txt";
    uint size = SIZE;
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 0;

    // Definition of testing variables
    uint repetitions;
    uint sizeArray[] = {50, 500, 1000};
    FILE *fptr;

    // Setup the testing variables
    if (TESTING) {
        repetitions = TEST_SIZE;
        fptr = fopen(path, "w");
        if (!fptr) {
            std::perror("File opening failed");
            return -1;
        }
    }
    else {
        repetitions = 1;
    }

    // events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Call to the Prim MST solver
    for (uint i = 0; i < repetitions; i++) {
        // Generation of the random graph
        if (TESTING) {
            size = sizeArray[i];
        }
        graph = new Graph(size, GPUEnabled);
        graph->randGraph(prob, true, maxWeight, fixedEng);

        // Test the printing procedure
        if (size < 15) {
            cout << "Printing the graph" << endl;
            graph->print(true);
        }

        cout << "Computing the MST solution for a graph of size " << size << endl;
        cudaEventRecord(start);
        mst *tree = primMST(size, variableEng, graph->getStruct());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        float milliseconds;
        cudaEventElapsedTime(&milliseconds, start, stop);
        float CPUtime = milliseconds / 1000.0;
        cout << "Time elapsed for CPU computation: " << CPUtime << endl;
        if (size < 15) {
            cout << "MST solution" << endl;
            for (int i = 0; i < size; ++i) {
                cout << i << ": ";
                for (int j = 0; j < tree->stree[i].size(); ++j) {
                    cout << tree->stree[i][j] << " ";
                }
                cout << endl;
            }
        }
        cout << "Total weight: " << tree->totalWeight << endl;
        cout << "\n\n\n\n";
        if (TESTING) {
            fprintf(fptr, "%d,%d,%f,\n", size, tree->totalWeight, CPUtime);
        }

        // Memory deallocation
        delete tree;
        delete graph;
        tree = NULL;
        graph = NULL;
    }

    if (TESTING) {
        fclose(fptr);
    }

    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));
    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/mstCPUPrim.cu -o mstCPUPrim
!./mstCPUPrim

## MST solution using the Prim solver with binary heap

In [64]:
%%cuda_group_save --name "CPUEfficientMst.cu" --group "CPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>
#include <string>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"
#include "mst.h"
#include "heap.h"

using namespace std;

mst *primMST (uint size, default_random_engine &eng, GraphStruct *str) {
    // Setup the mst
    mst *tree = new mst();
    tree->stree = new vector<int>[size];
    tree->totalWeight = 0;

    // Select a starting node
    vector<node> Remaining;
    uniform_real_distribution<> randR(0.0, 1.0);
    node randV;
    randV = (node)(randR(eng) * size);
    cout << "Source for the MST: " << randV << endl;

    // Initializing the heap structure
    Heap *heap = new Heap(size);
    for (int i = 0; i < size; ++i) {
        int offset = str->isNeighbor(randV, i);
        if (offset >= 0) {
            heap->insert(randV, i, offset, str->getWeight(randV, offset));
        }
        else if (i != randV) {
            heap->insert(UINT_MAX, i, UINT_MAX, INT_MAX);
        }
    }

    // While the heap is not empty
    while (!heap->isEmpty()) {
        edge candidateEdge = heap->pop();
        node destinationCandidate = candidateEdge.destination;

        for (uint i = 0; i < str->deg(destinationCandidate); i++) {
            node neigh = str->getNeigh(destinationCandidate, i);
            node weight = str->getWeight(destinationCandidate, i);
            uint pos = heap->getPosition(neigh);
            bool inMst = false;

            if (pos == UINT_MAX) {
                inMst = true;
            }

            if (!inMst && weight < heap->getKey(pos).weight) {
                heap->keyDecrease(pos, destinationCandidate, weight);
            }
        }

        // Add the newfound edge to the mst
        tree->totalWeight += candidateEdge.weight;
        tree->stree[candidateEdge.source].push_back(candidateEdge.destination);
    }

    delete heap;
    heap = NULL;

    return tree;
}

int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine fixedEng(FIXED_SEED);
    std::default_random_engine variableEng(rd());
    Graph *graph;
    //string spath = LOGPATH + to_string(SIZE) + "_" + to_string(FIXED_SEED) + ".txt";
    //char path[] = spath.c_str();
    uint size = SIZE;
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 0;

    //cout << path << endl;
    //return 0;

    // Definition of testing variables
    uint repetitions;
    uint sizeArray[] = {50, 500, 1000};
    FILE *fptr;

    // Setup the testing variables
    if (TESTING) {
        repetitions = TEST_SIZE;
        //fptr = fopen(path, "w");
        if (!fptr) {
            std::perror("File opening failed");
            return -1;
        }
    }
    else {
        repetitions = 1;
    }

    // events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Call to the Prim MST solver
    for (uint i = 0; i < repetitions; i++) {
        // Generation of the random graph
        if (TESTING) {
            size = sizeArray[i];
        }
        graph = new Graph(size, GPUEnabled);
        graph->randGraph(prob, true, maxWeight, fixedEng);

        // Test the printing procedure
        if (size < 15) {
            cout << "Printing the graph" << endl;
            graph->print(true);
        }
        cudaEventRecord(start);
        cout << "Computing the MST solution for a graph of size " << size << endl;
        mst *tree = primMST(size, variableEng, graph->getStruct());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        float milliseconds;
        cudaEventElapsedTime(&milliseconds, start, stop);
        float CPUtime = milliseconds / 1000.0;
        cout << "Time elapsed for CPU computation: " << CPUtime << endl;
        cout << "Total weight: " << tree->totalWeight << endl;
        cout << "\n\n\n\n";
        if (TESTING) {
            fprintf(fptr, "%d,%d,\n", size, tree->totalWeight);
        }

        // Memory deallocation
        delete graph;
        delete tree;
        tree = NULL;
        graph = NULL;
    }

    if (TESTING) {
        fclose(fptr);
    }

    return 0;
}

In [65]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75  GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/CPU/CPUEfficientMst.cu -o CPUEfficientMst
!./CPUEfficientMst

src/CPU/CPUEfficientMst.cu(103): warning #549-D: variable "fptr" is used before its value is set
          if (!fptr) {
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

src/CPU/CPUEfficientMst.cu(103): warning #549-D: variable "fptr" is used before its value is set
          if (!fptr) {
               ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Computing the MST solution for a graph of size 20000
Source for the MST: 11082
Time elapsed for CPU computation: 6.59369
Total weight: 111239






# GPU zone

## Hybrid approach

The code below uses an hybrid approach where a part of the code is working with the GPU and a part of the code is using the CPU, namely:

- Finding the cheapest edge for every node is implemented using a GPU kernel
- The scan operation is implemented with a CPU procedure because it's more efficient than the naive GPU counterpart
- The removal of mirrored edges is implemented using a GPU kernel
- Finding the connected components is done through a recursive GPU kernel
- Calculating the outdegrees for the supervertices is done with a GPU kernel
- The new graph allocation is done through a CPU function

In [56]:
%%cuda_group_save --name "mstHBD.cu" --group "GPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph_d.h"
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

/*****
* Device function that gets the degree of a certain node
* @param str - The structure of the graph
* @param i - The node we are interested in
*****/
__device__ node d_deg (GraphStruct *str, node i) {
    return str->cumDegs[i + 1] - str->cumDegs[i];
}

/*****
* Device function that gets the weight of a certain edge
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ int d_getWeight (GraphStruct *str, node i, uint offset) {
    return str->weights[str->cumDegs[i] + offset];
}

/*****
* Device function that gets the neighbour of a certain node
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ node d_getNeigh (GraphStruct *str, node i, uint offset) {
    return str->neighs[str->cumDegs[i] + offset];
}

__device__ uint d_getRoot (uint i, uint *d_flag, uint *d_colors) {
    return max(0, d_flag[d_colors[i]] - 1);
}

uint getRoot (uint i, uint *flag, uint *colors) {
    return max(0, flag[colors[i]] - 1);
}


/*****
* Kernel that finds the cheapest edge in the adjacency list of every node
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void findCheapest (GraphStruct *str, uint *d_candidates) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    // Initialize the minimum value
    uint minimum = UINT_MAX;
    int minimumWeight = INT_MAX;

    // Find the cheapest edge in each adjacency list
    for (uint i = 0; i < d_deg(str, idx); i++) {
        int edgeWeight = d_getWeight(str, idx, i);
        if (edgeWeight < minimumWeight) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
        else if (edgeWeight == minimumWeight &&
                 d_getNeigh(str, idx, i) < d_getNeigh(str, idx, minimum)) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
    }

    // Update the return vector
    d_candidates[idx] = minimum;
}


/*****
* Kernel that removes the mirrored edges from the graph. A mirrored edge is
* simply an edge pointing from the source to the destination and vice versa in
* an oriented graph, the removal logic is to cut the edge with the lowest source
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void mirroredEdgesRemoval (GraphStruct *str, uint *d_candidates, int *d_weight) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint destinationOffset = d_candidates[idx];
    node destination = d_getNeigh(str, idx, destinationOffset);
    if (idx < destination) {
        uint sourceOffset = d_candidates[destination];
        node destinationNeigh = d_getNeigh(str, destination, sourceOffset);

        // The vertex cannot be a candidate anymore because it would create a cycle
        if (destinationNeigh == idx) {
            d_candidates[idx] = UINT_MAX;
        }
    }

    if (d_candidates[idx] != UINT_MAX) {
        atomicAdd(d_weight, d_getWeight(str, idx, d_candidates[idx]));
    }
}


/*****
* Helper device function that recursively colors the nodes of the graph
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
* @param i - The index of the node to be colored
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__device__ uint *d_recursiveColorationHelper (GraphStruct *str, uint *d_candidates, node i, uint *d_colors) {
    uint color = UINT_MAX;
    if (d_candidates[i] == UINT_MAX) {
        color = i;
    }
    else {
        node neigh = d_getNeigh(str, i, d_candidates[i]);
        color = d_recursiveColorationHelper(str, d_candidates, neigh, d_colors)[neigh];
    }

    if (color != UINT_MAX) {
        d_colors[i] = color;
    }
    return d_colors;
}


/*****
* Kernel that recognizes the connected components in the graph and colors them
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__global__ void colorationProcess(GraphStruct *str, uint *d_candidates, uint *d_colors) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    d_recursiveColorationHelper(str, d_candidates, idx, d_colors);
}


__global__ void cumulatedDegreeUpdate(GraphStruct *str, uint *d_cumDegs, uint *d_colors, uint *d_flag) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node svSuccessor = d_getRoot(idx, d_flag, d_colors) + 1;
    uint sum = 0;

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            sum++;
        }
    }

    atomicAdd(&(d_cumDegs[svSuccessor]), sum);
}


uint *CPUScan(uint *input, uint size) {
    for (uint i = 1; i < size; i++) {
        input[i] = input[i - 1] + input[i];
    }
    return input;
}


int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine eng(FIXED_SEED);
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 1;
    Graph *graphPointer;
    Graph graph(SIZE, GPUEnabled);
    graphPointer = &graph;
  	graphPointer->randGraph(prob, true, maxWeight, eng);
    /**************************************************/


    // Checking if the random graph is connected
    if (!graphPointer->isConnected()) {
        cout << "The graph is not connected" << endl;
        return -1;
    }
    /************/


    uint iterations = 0;


    // Configuration of the GPU kernel
    uint blockDim = BLOCK_SIZE;
    uint *candidates;
    /***************/


    // Events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float milliseconds;
    float spliTime = 0;
    float totalTime = 0;
    /******************/


    // Variables calculating the MST weight
    int mstWeight = 0;
    int *d_mstWeight;
    CHECK(cudaMalloc((void **)&d_mstWeight, sizeof(int)));
    CHECK(cudaMemcpy(d_mstWeight, &mstWeight, sizeof(int), cudaMemcpyHostToDevice));
    /******************************************************************************/


    // Main block of the algorithm
    while (graphPointer->getStruct()->nodeSize > 1) {
        // Initialization of the variables associated with the graph
        GraphStruct *str = graphPointer->getStruct();
        uint size = str->nodeSize;
        uint edgeSize = str->edgeSize;
        cout << "Processing a graph of size: " << size << " with " << edgeSize << " edges.\n\n";
        uint gridDim = (size + blockDim - 1) / blockDim;
        if (DEBUGGING && size < 15 && str->edgeSize < 100) {
            graphPointer->print(true);
            print_d<<<1, 1>>>(str, 1);
            CHECK(cudaDeviceSynchronize());
        }
        candidates = new uint[size];
        /**************************/

        // First setp of the algorithm
        uint *d_candidates;
        CHECK(cudaMalloc((void**)&d_candidates, (size) * sizeof(uint)));
        CHECK(cudaMemset(d_candidates, 0, (size) * sizeof(uint)));
        cout << "Launching kernel FIND CHEAPEST -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        findCheapest<<<gridDim, blockDim>>>(str, d_candidates);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Finding the cheapest edge for every vertex took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex
        if (DEBUGGING && size < 15) {
            cout << "The cheapest edge for every vertex" << endl;
            CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> " << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
            }
            cout << "\n\n\n";
        }
        /*******************/



        // Second step of the algorithm
        cout << "Launching kernel MIRRORED EDGES REMOVAL -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        mirroredEdgesRemoval<<<gridDim, blockDim>>>(str, d_candidates, d_mstWeight);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(&mstWeight, d_mstWeight, sizeof(int), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex update
        if (DEBUGGING && size < 15) {
            cout << "Update of the cheapest edge for every vertex" << endl;
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> ";
                if (candidates[i] != UINT_MAX) {
                    cout << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
                }
                else {
                    cout << "NULL" << endl;
                }
            }
            printf ("%d\n", mstWeight);
        }
        /*****************************/
        printf ("%d\n", mstWeight);



        // Third step of the algorithm
        cout << "Launching kernel COLORATION PROCESS -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        // Initialize the color array
        uint *colors = new uint[size];
        uint *d_colors;
        CHECK(cudaMalloc((void**)&d_colors, size * sizeof(uint)));
        CHECK(cudaMemset(d_colors, UINT_MAX, size * sizeof(uint)));
        /*********************************************************/

        cudaEventRecord(start);
        colorationProcess<<<gridDim, blockDim>>>(str, d_candidates, d_colors);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(colors, d_colors, size * sizeof(uint), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;

        // Print the coloring
        if (DEBUGGING) {
            uint *checkColoring = new uint[size];

            for (uint i = 0; i < size; i++) {
                checkColoring[i] = 0;
            }

            for (uint i = 0; i < size; i++) {
                checkColoring[colors[i]]++;
            }

            uint nonZeroColors = 0;
            for (uint i = 0; i < size; i++) {
                if (checkColoring[i] != 0) {
                    cout << "color " << i << "\t" << checkColoring[i] << endl;
                    nonZeroColors++;
                }
            }

            cout << "There is a total of " << nonZeroColors << " colors" << endl;

            cout << "\n\n\n";
        }
        /*******************/

        /**
         * If the coloring coming out of the last kernel contains only one color
         * then it means that the edge added in the last step was the one needed
         * to merge the partial trees
         **/
        uint color = colors[0];
        bool uniqueColor = true;
        for (uint i = 1; i < size; i++) {
            if (colors[i] != color) {
                uniqueColor = false;
                break;
            }
        }
        if (uniqueColor) {
            cout << "THE CALCULATION OF THE MST IS COMPLETE\n";
            cout << "THE MST WEIGHT IS: " << mstWeight << endl;
            printf("Total elapsed time: %.5f seconds\n\n", totalTime);

            // Cuda memory deallocation
            CHECK(cudaEventDestroy(start));
            CHECK(cudaEventDestroy(stop));
            CHECK(cudaFree(d_candidates));
            CHECK(cudaFree(d_colors));

            // Host memory deallocation
            delete[] candidates;
            delete[] colors;

            return 0;
        }
        /***********/



        // Fourth step of the algorithm
        cout << "Launching a round of CPU scan\n\n" << endl;
        uint *flag = new uint[size];
        for (uint i = 0; i < size; i++) {
            flag[i] = (colors[i] == i) ? 1 : 0;
        }
        cudaEventRecord(start);
        flag = CPUScan(flag, size);
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the prefix sum of the auxiliary flag array took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        if (DEBUGGING) {
            for (uint i = 0; i < size; i++) {
                if (colors[i] == i) {
                    cout << "Mapping color " << i << " to " << getRoot(i, flag, colors) << endl;
                }
            }
        }

        uint *d_flag;

        CHECK(cudaMalloc((void**)&d_flag, (size) * sizeof(uint)));
        CHECK(cudaMemcpy(d_flag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));
        /*****************************************************************************/



        // Fifth step of the algorithm

        // Allocating resources for the new cumulated degrees array
        uint newNodeSize = flag[size - 1];
        uint cumDegSize = newNodeSize + 1;
        uint *cumDegs = new uint[cumDegSize];
        uint *d_cumDegs;
        CHECK(cudaMalloc((void**)&d_cumDegs, (cumDegSize) * sizeof(uint)));
        CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));
        /***********************************************************/

        cout << "Launching kernel CUMULATED DEGREE UPDATE -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        cudaEventRecord(start);
        cumulatedDegreeUpdate<<<gridDim, blockDim>>>(str, d_cumDegs, d_colors, d_flag);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the computation of the cumulated degrees took: %.5f seconds\n\n", spliTime);
        CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

        // ~Debugging~ looking for errors in the cumulated degrees array
        if (DEBUGGING) {
            uint *checkDegs = new uint[cumDegSize];
            for (uint i = 0; i < cumDegSize; i++) {
                checkDegs[i] = 0;
            }
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node svSuccessor = getRoot(i, flag, colors) + 1;
                uint sum = 0;

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        sum++;
                    }
                }

                checkDegs[svSuccessor] += sum;
            }
            for (uint i = 0; i < cumDegSize; i++) {
                if (checkDegs[i] != cumDegs[i]) {
                    cout << i << " " << checkDegs[i] << " " << cumDegs[i] << endl;
                    return -1;
                }
            }
            cout << "The CPU check vector and the GPU computed one are the same" << endl;
            delete[] checkDegs;
        }
        /*********************/


        // Perform another prefix sum on the cumDegrees array
        cout << "Launching a round of CPU scan\n\n" << endl;
        cudaEventRecord(start);
        cumDegs = CPUScan(cumDegs, cumDegSize);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("Doing the scan of the new cumDegs array took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;

        // ~Debugging~ print the results of the scan operation on the cum degrees array
        if (DEBUGGING) {
            uint j = 0;
            for (uint i = 0; i < cumDegSize; i++) {
                cout << cumDegs[i] << "   ";
                j++;
                if (j == 10) {
                    cout << endl;
                    j = 0;
                }
            }
        }
        /****************/


        // Allocating space for the arrays in the newly contracted graph
        uint newEdgeSize = cumDegs[cumDegSize - 1];
        node *newNeighs = new node[newEdgeSize];
        uint *newWeights = new uint[newEdgeSize];
        /***************************************/

        // Copy the contents of cumDegs into a new array
        uint *cCumDegs = new uint[cumDegSize];
        for (uint i = 0; i < cumDegSize; i++) {
            cCumDegs[i] = cumDegs[i];
        }

        cudaEventRecord(start);
        for (uint i = 0; i < size; i++) {
            uint color = colors[i];
            node superVertex = getRoot(i, flag, colors);

            for (uint j = 0; j < str->deg(i); j++) {
                node neigh = str->getNeigh(i, j);
                uint neighColor = colors[neigh];

                if (color != neighColor) {
                    int weight = str->getWeight(i, j);
                    uint position = cCumDegs[superVertex];
                    newNeighs[position] = getRoot(neigh, flag, colors);
                    newWeights[position] = weight;
                    cCumDegs[superVertex]++;
                }
            }
        }
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("The construction of the new neighbour and weight arrays took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;



        // Reconstructing the graph
        graphPointer->copyConstructor(newNodeSize, newEdgeSize, newNeighs, newWeights, cumDegs);
        printf("----------------------------------\n\n");
        /***********************************************/



        // Updating the iteration information
        totalTime += spliTime;
        iterations++;
        /***********/


        // Cuda memory deallocation
        CHECK(cudaFree(d_candidates));
        CHECK(cudaFree(d_colors));
        CHECK(cudaFree(d_flag));
        CHECK(cudaFree(d_cumDegs));
        /*************************/

        // Host memory deallocation
        delete[] candidates;
        delete[] colors;
        delete[] flag;
        delete[] cumDegs;
        delete[] newNeighs;
        delete[] newWeights;
        delete[] cCumDegs;
        /****************/
    }

    printf("Total elapsed time: %.5f seconds\n\n", totalTime);
    printf("The calculation of the MST took %d iterations\n\n", iterations);
    printf("The total weight of the tree is %d\n", mstWeight);


    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));

    return 0;
}

In [66]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75 GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/GPU/mstHBD.cu -o mstHBD
!./mstHBD

ptxas warning : Stack size for entry function '_Z17colorationProcessP11GraphStructPjS1_' cannot be statically determined
Processing a graph of size: 20000 with 199990050 edges.

Launching kernel FIND CHEAPEST -- (1024, 1, 1) -- (20, 1, 1)
Finding the cheapest edge for every vertex took: 0.21779 seconds

Launching kernel MIRRORED EDGES REMOVAL -- (1024, 1, 1) -- (20, 1, 1)
Removing the mirrored edges required: 0.05990 seconds

80484
Launching kernel COLORATION PROCESS -- (1024, 1, 1) -- (20, 1, 1)
Removing the mirrored edges required: 0.00007 seconds

Launching a round of CPU scan


Doing the prefix sum of the auxiliary flag array took: 0.00007 seconds

Launching kernel CUMULATED DEGREE UPDATE -- (1024, 1, 1) -- (20, 1, 1)
Doing the computation of the cumulated degrees took: 0.01221 seconds

Launching a round of CPU scan


Doing the scan of the new cumDegs array took: 0.00003 seconds

The construction of the new neighbour and weight arrays took: 5.90433 seconds

------------------------

## GPU only approach

The code below is technically thought to work only on the GPU, the statement holds true with an asterisk (*)

- A naive extraction of the methods calculating the graph contraction and the new cumulated degrees array has been done

- The methods are not efficient to utilize the gpu's vector unit infact the methods that hurt the performance the most are the one looking for the cheapest edges and the one building the new arrays for the contracted graph

- I wrote a function that is able to compute the scan in three phases:
  - Compute the scan of the first part of the vector on the GPU using a work efficient technique (based on trees) [look at parprefix], this is computed on a window that is the maximum possible multiple of the smem window.
  - Compute the scan of the rest of the array (if it's there is less than a smem window) on the CPU (*)
  - Compute the scan of the auxiliary array containing the sum of the elements inside the block (*)
  - Compute a final sum of the elements inside block j with the element in position j inside the new vector

I decided to implement it this way because if the array is very small then computing the sum through a scan on the GPU is actually less efficient than doing it on the CPU.

Finally the upgrade of this new version is not incredible but it's there, as an example:

SIZE - 20000

DENSITY - 0.5

HBD_TIME - 47.98

GPU_TIME - 41.67


In [69]:
%%cuda_group_save --name "mstGPUE.cu" --group "GPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph_d.h"
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

/*****
* Device function that gets the degree of a certain node
* @param str - The structure of the graph
* @param i - The node we are interested in
*****/
__device__ node d_deg (GraphStruct *str, node i) {
    return str->cumDegs[i + 1] - str->cumDegs[i];
}

/*****
* Device function that gets the weight of a certain edge
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ int d_getWeight (GraphStruct *str, node i, uint offset) {
    return str->weights[str->cumDegs[i] + offset];
}

/*****
* Device function that gets the neighbour of a certain node
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ node d_getNeigh (GraphStruct *str, node i, uint offset) {
    return str->neighs[str->cumDegs[i] + offset];
}

__device__ uint d_getRoot (uint i, uint *d_flag, uint *d_colors) {
    return max(0, d_flag[d_colors[i]]);
}

uint getRoot (uint i, uint *flag, uint *colors) {
    return max(0, flag[colors[i]]);
}


/*****
* Kernel that finds the cheapest edge in the adjacency list of every node
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void findCheapest (GraphStruct *str, uint *d_candidates) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    // Initialize the minimum value
    uint minimum = UINT_MAX;
    int minimumWeight = INT_MAX;

    // Find the cheapest edge in each adjacency list
    for (uint i = 0; i < d_deg(str, idx); i++) {
        int edgeWeight = d_getWeight(str, idx, i);
        if (edgeWeight < minimumWeight) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
        else if (edgeWeight == minimumWeight &&
                 d_getNeigh(str, idx, i) < d_getNeigh(str, idx, minimum)) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
    }

    // Update the return vector
    d_candidates[idx] = minimum;
}


/*****
* Kernel that removes the mirrored edges from the graph. A mirrored edge is
* simply an edge pointing from the source to the destination and vice versa in
* an oriented graph, the removal logic is to cut the edge with the lowest source
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void mirroredEdgesRemoval (GraphStruct *str, uint *d_candidates, int *d_weight) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint destinationOffset = d_candidates[idx];
    node destination = d_getNeigh(str, idx, destinationOffset);
    if (idx < destination) {
        uint sourceOffset = d_candidates[destination];
        node destinationNeigh = d_getNeigh(str, destination, sourceOffset);

        // The vertex cannot be a candidate anymore because it would create a cycle
        if (destinationNeigh == idx) {
            d_candidates[idx] = UINT_MAX;
        }
    }

    if (d_candidates[idx] != UINT_MAX) {
        atomicAdd(d_weight, d_getWeight(str, idx, d_candidates[idx]));
    }
}


/*****
* Helper device function that recursively colors the nodes of the graph
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
* @param i - The index of the node to be colored
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__device__ uint *d_recursiveColorationHelper (GraphStruct *str, uint *d_candidates, node i, uint *d_colors) {
    uint color = UINT_MAX;
    if (d_candidates[i] == UINT_MAX) {
        color = i;
    }
    else {
        node neigh = d_getNeigh(str, i, d_candidates[i]);
        color = d_recursiveColorationHelper(str, d_candidates, neigh, d_colors)[neigh];
    }

    if (color != UINT_MAX) {
        d_colors[i] = color;
    }
    return d_colors;
}


/*****
* Kernel that recognizes the connected components in the graph and colors them
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__global__ void colorationProcess(GraphStruct *str, uint *d_candidates, uint *d_colors) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    d_recursiveColorationHelper(str, d_candidates, idx, d_colors);
}


/**
* Kernel that computes the prefix sun of the auxiliary flag array, code taken
* from the lectures and rearranged to follow the logic required for the
* implementation of the algorithm
* @param str - The structure of the graph
* @param d_colors - The device-level array of colors assigned to each vertex
* @param d_flag - The device-level array of flag values for the prefix sum.
*                 d_flag[i] = 1 if and only if the vertex in position i has the
*                 same color as the index => I recognize the root of the
*                 connected component
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void blockScan(uint *size, uint *input, uint *auxiliarVector) {
   __shared__ uint smem[BLOCK_SIZE];
   uint tid = threadIdx.x;
   uint idx = tid + blockIdx.x * blockDim.x;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   // Load input into shared memory.
   smem[tid] = input[idx];
   __syncthreads();

   // do recursive sums
   for (uint d = 1; d < BLOCK_SIZE; d *= 2) {
      if (tid >= d)
         smem[tid] += smem[tid - d];
      __syncthreads();
   }
   input[idx] = smem[tid];
   if (idx == ((blockIdx.x + 1) * blockDim.x - 1))
      auxiliarVector[blockIdx.x] = input[idx];
}


/**
* Auxiliary kernel that computes the sum of the partial values calculated using
* the blockScan kernel
* str - The structure of the graph
* @param d_flag - The device-level array containing intermediate sums obtained
*                 with the previous kernel
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void sumBlockScan(uint *size, uint *input, uint *auxiliarVector) {
   uint idx = threadIdx.x + blockIdx.x * blockDim.x;  // global index 0:n-1
   uint s = 0;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   if (blockIdx.x == 0) return;

   for (uint j = 0; j < blockIdx.x; j++)
      s += auxiliarVector[j];

   // add term to input
   input[idx] += s;
}

//*** SCAN FUNCTIONS ***//

__global__ void prescan(uint *g_odata, uint *g_idata, uint *aux, int n, int smemSize)
{
  extern __shared__ int temp[];// allocated on invocation
  int thid = threadIdx.x;
  int offset = 1;
  int idx = blockIdx.x * blockDim.x + thid;

  temp[2*thid] = g_idata[2*idx]; // load input into shared memory
  temp[2*thid+1] = g_idata[2*idx+1];

  for (int d = n>>1; d > 0; d >>= 1) // build sum in place up the tree
  {
    __syncthreads();
    if (thid < d)
    {
      int ai = offset*(2*thid+1)-1;
      int bi = offset*(2*thid+2)-1;
      if (bi < smemSize && ai < smemSize) {
        temp[bi] += temp[ai];
      }
    }
    offset *= 2;
  }

  if (thid == 0)
  {
    aux[blockIdx.x] = temp[smemSize - 1];
    temp[smemSize - 1] = 0;
  } // clear the last element

  for (int d = 1; d < n; d *= 2) // traverse down tree & build scan
  {
    __syncthreads();
    if (thid < d && offset > 0)
    {
      int ai = offset*(2*thid+1)-1;
      int bi = offset*(2*thid+2)-1;
      if (bi < smemSize && ai < smemSize) {
        int t = temp[ai];
        temp[ai] = temp[bi];
        temp[bi] += t;
      }
    }
    offset >>= 1;
  }


  __syncthreads();
  if (idx <= (n / 2) - 1) {
      g_odata[2*idx] = temp[2*thid]; // write results to device memory
      g_odata[2*idx+1] = temp[2*thid+1];
  }
}

void cpuScan(uint *array, int start, int end) {
    if (end - start <= 1) {
        return;
    }

    int temp = array[start + 1];
    array[start + 1] = array[start];
    array[start] = 0;

    for (uint i = start + 1; i < end - 1; i++) {
        int sum = array[i] + temp;
        temp = array[i + 1];
        array[i + 1] = sum;
    }
}


__global__ void final_sum(uint *g_odata, uint *aux, uint n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  if (blockIdx.x == 0 || 2 * idx >= n) {
      return;
  }

  //printf("%d: ls - %d  rs - %d  aux - %d\n", idx, g_odata[2 * idx], g_odata[2 * idx + 1], aux[blockIdx.x - 1]);

  if (2 * idx == n - 1) {
      g_odata[2 * idx] += aux[blockIdx.x];
      return;
  }
  g_odata[2 * idx] += aux[blockIdx.x];
  g_odata[2 * idx + 1] += aux[blockIdx.x];
}

//****************//


__global__ void cumulatedDegreeUpdate(GraphStruct *str, uint *d_cumDegs, uint *d_colors, uint *d_flag) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node svSuccessor = d_getRoot(idx, d_flag, d_colors);
    uint sum = 0;

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            sum++;
        }
    }

    atomicAdd(&(d_cumDegs[svSuccessor]), sum);
}


__global__ void graphContraction(GraphStruct *str, uint *d_colors, uint *d_flag,
                                 uint *d_cumDegs, node *d_newNeighs, uint *d_newWeights) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node superVertex = d_getRoot(idx, d_flag, d_colors);

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            int weight = d_getWeight(str, idx, i);
            uint position = atomicAdd(&(d_cumDegs[superVertex]), 1);
            d_newNeighs[position] = d_getRoot(neigh, d_flag, d_colors);
            d_newWeights[position] = weight;
        }
    }
}


int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine eng(FIXED_SEED);
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 1;
    Graph *graphPointer;
    Graph graph(SIZE, GPUEnabled);
    graphPointer = &graph;
  	graphPointer->randGraph(prob, true, maxWeight, eng);
    /**************************************************/


    // Checking if the random graph is connected
    if (!graphPointer->isConnected()) {
        cout << "The graph is not connected" << endl;
        return -1;
    }
    /************/


    uint iterations = 0;


    // Configuration of the GPU kernel
    uint blockDim = BLOCK_SIZE;
    uint *candidates;
    /***************/


    // Events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float milliseconds;
    float spliTime = 0;
    float totalTime = 0;
    /******************/


    // Variables calculating the MST weight
    int mstWeight = 0;
    int *d_mstWeight;
    CHECK(cudaMalloc((void **)&d_mstWeight, sizeof(int)));
    CHECK(cudaMemcpy(d_mstWeight, &mstWeight, sizeof(int), cudaMemcpyHostToDevice));
    /******************************************************************************/


    // Main block of the algorithm
    while (graphPointer->getStruct()->nodeSize > 1) {
        // Initialization of the variables associated with the graph
        GraphStruct *str = graphPointer->getStruct();
        uint size = str->nodeSize;
        uint edgeSize = str->edgeSize;
        cout << "Processing a graph of size: " << size << " with " << edgeSize << " edges.\n\n";
        uint gridDim = (size + blockDim - 1) / blockDim;
        if (DEBUGGING && size < 15 && str->edgeSize < 100) {
            graphPointer->print(true);
            print_d<<<1, 1>>>(str, 1);
            CHECK(cudaDeviceSynchronize());
        }
        candidates = new uint[size];
        /******************************/

        // First setp of the algorithm
        uint *d_candidates;
        CHECK(cudaMalloc((void**)&d_candidates, (size) * sizeof(uint)));
        CHECK(cudaMemset(d_candidates, 0, (size) * sizeof(uint)));
        cout << "Launching kernel FIND CHEAPEST -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        findCheapest<<<gridDim, blockDim>>>(str, d_candidates);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Finding the cheapest edge for every vertex took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex
        if (DEBUGGING && size < 15) {
            cout << "The cheapest edge for every vertex" << endl;
            CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> " << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
            }
            cout << "\n\n\n";
        }
        /*******************/



        // Second step of the algorithm
        cout << "Launching kernel MIRRORED EDGES REMOVAL -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        mirroredEdgesRemoval<<<gridDim, blockDim>>>(str, d_candidates, d_mstWeight);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(&mstWeight, d_mstWeight, sizeof(int), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex update
        if (DEBUGGING && size < 15) {
            cout << "Update of the cheapest edge for every vertex" << endl;
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> ";
                if (candidates[i] != UINT_MAX) {
                    cout << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
                }
                else {
                    cout << "NULL" << endl;
                }
            }
            printf ("%d\n", mstWeight);
        }
        /*****************************/

        cout << "The MST weight at the end of iteration " << iterations + 1 << " is: " << mstWeight << endl;



        // Third step of the algorithm
        cout << "Launching kernel COLORATION PROCESS -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        // Initialize the color array
        uint *colors = new uint[size];
        uint *d_colors;
        CHECK(cudaMalloc((void**)&d_colors, size * sizeof(uint)));
        CHECK(cudaMemset(d_colors, UINT_MAX, size * sizeof(uint)));
        /**************************************************/

        cudaEventRecord(start);
        colorationProcess<<<gridDim, blockDim>>>(str, d_candidates, d_colors);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(colors, d_colors, size * sizeof(uint), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;

        // Print the coloring
        if (DEBUGGING) {
            uint *checkColoring = new uint[size];

            for (uint i = 0; i < size; i++) {
                checkColoring[i] = 0;
            }

            for (uint i = 0; i < size; i++) {
                checkColoring[colors[i]]++;
            }

            uint nonZeroColors = 0;
            for (uint i = 0; i < size; i++) {
                if (checkColoring[i] != 0) {
                    nonZeroColors++;
                }
            }

            cout << "There is a total of " << nonZeroColors << " colors" << endl;

            cout << "\n\n\n";
        }
        /*******************/

        /**
         * If the coloring coming out of the last kernel contains only one color
         * then it means that the edge added in the last step was the one needed
         * to merge the partial trees
         **/
        uint color = colors[0];
        bool uniqueColor = true;
        for (uint i = 1; i < size; i++) {
            if (colors[i] != color) {
                uniqueColor = false;
                break;
            }
        }
        if (uniqueColor) {
            cout << "THE CALCULATION OF THE MST IS COMPLETE\n";
            cout << "THE MST WEIGHT IS: " << mstWeight << endl;
            printf("Total elapsed time: %.5f seconds\n\n", totalTime);

            // Cuda memory deallocation
            CHECK(cudaEventDestroy(start));
            CHECK(cudaEventDestroy(stop));
            CHECK(cudaFree(d_candidates));
            CHECK(cudaFree(d_colors));

            // Host memory deallocation
            delete[] candidates;
            delete[] colors;

            return 0;
        }
        /***********/




        // Fourth step of the algorithm
        cout << "Doing a round of scan on the flag vector, size: " << size << endl;
        uint *flag = new uint[size];
        uint *cFlag = new uint[size];
        for (uint i = 0; i < size; i++) {
            flag[i] = (colors[i] == i) ? 1 : 0;
            cFlag[i] = flag[i];
        }
        uint *d_flag;
        uint smemSize = 2 * blockDim;

        CHECK(cudaMalloc((void**)&d_flag, (size) * sizeof(uint)));

        if (size < smemSize) {
            cout << "Resorting to a round of CPU scan" << endl;
            cpuScan(flag, 0, size);
            CHECK(cudaMemcpy(d_flag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));
        }
        else {
            uint *d_aux, *aux, *d_ogFlag;

            uint numSmemBlock = size / smemSize;
            uint numBlock = (size + blockDim - 1) / blockDim;
            uint gpuScanSize = numSmemBlock * smemSize;
            uint residualSize = size - gpuScanSize;

            aux = (uint *) malloc((numSmemBlock + 1) * sizeof(uint));

            CHECK(cudaMalloc((void **) &d_aux, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMalloc((void **) &d_ogFlag, size * sizeof(uint)));

            CHECK(cudaMemcpy(d_ogFlag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));

            CHECK(cudaMemset(d_aux, 0, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMemset(d_flag, 0, (size) * sizeof(uint)));

            printf("\n  block scan...\n");

            uint smem = smemSize * sizeof(uint);
            printf("\n  first prescan procedure on the Device: %d elements...\n", gpuScanSize);
            cudaEventRecord(start);
            prescan<<<  numSmemBlock, blockDim, smem >>>(d_flag, d_ogFlag, d_aux, size, smemSize);
            printf("\n  second scan procedure on the Host: %d elements...\n", residualSize);
            cpuScan(flag, gpuScanSize, size);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            float milliseconds;
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime = milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the contents of the aux array into Host memory and perform another scan
            printf("\n  third scan procedure on the Host: %d elements...\n", numSmemBlock);
            CHECK(cudaMemcpy(aux, d_aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyDeviceToHost));
            cudaEventRecord(start);
            cpuScan(aux, 0, numSmemBlock + 1);
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the portions of the array computed on the Host to Device memory
            CHECK(cudaMemcpy(d_aux, aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyHostToDevice));
            CHECK(cudaMemcpy(&(d_flag[gpuScanSize]), &(flag[gpuScanSize]), residualSize * sizeof(uint), cudaMemcpyHostToDevice));

            printf("\n  final summation procedure...\n");
            cudaEventRecord(start);
            final_sum<<< numBlock, blockDim >>>(d_flag, d_aux, size);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n\n", milliseconds / 1000.0);

            printf("\nTotal elapsed time:   %.5f (sec)\n", spliTime);

            totalTime += spliTime;
            CHECK(cudaMemcpy(flag, d_flag, (size) * sizeof(uint), cudaMemcpyDeviceToHost));

            free(aux);
            CHECK(cudaFree(d_aux));
            CHECK(cudaFree(d_ogFlag));
        }

        if (DEBUGGING) {
            for (uint i = 1; i < size; i++) {
                cFlag[i] += cFlag[i - 1];
            }

            for (uint i = 0; i < size - 1; i++) {
                if (cFlag[i] != flag[i + 1]) {
                    cout << "I due array sono diversi in posizione " << i << endl;
                    return -1;
                }
            }
        }
        delete[] cFlag;
        cout << "The contracted graph will contain " << flag[size - 1] << " supervertices\n\n" << endl;





        // Fifth step of the algorithm

        // Allocating resources for the new cumulated degrees array
        uint newNodeSize = flag[size - 1];
        uint cumDegSize = newNodeSize + 1;
        uint *cumDegs = new uint[cumDegSize];
        uint *d_cumDegs;
        CHECK(cudaMalloc((void**)&d_cumDegs, (cumDegSize) * sizeof(uint)));
        CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));
        /***********************************/

        cout << "Launching kernel CUMULATED DEGREE UPDATE -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        cudaEventRecord(start);
        cumulatedDegreeUpdate<<<gridDim, blockDim>>>(str, d_cumDegs, d_colors, d_flag);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the computation of the cumulated degrees took: %.5f seconds\n\n", spliTime);
        CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

        // ~Debugging~ looking for errors in the cumulated degrees array
        if (DEBUGGING) {
            uint *checkDegs = new uint[cumDegSize];
            for (uint i = 0; i < cumDegSize; i++) {
                checkDegs[i] = 0;
            }
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node svSuccessor = getRoot(i, flag, colors);
                uint sum = 0;

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        sum++;
                    }
                }

                checkDegs[svSuccessor] += sum;
            }
            for (uint i = 0; i < cumDegSize; i++) {
                if (cumDegs[i] != checkDegs[i]) {
                    cout << i << ": cumDegs - " << cumDegs[i] << "\tcheckDegs - " << checkDegs[i] << endl;
                    return -1;
                }
            }
            cout << "The CPU check vector and the GPU computed one are the same\n\n" << endl;
            delete[] checkDegs;
        }
        /********************/




        // Perform another prefix sum on the cumDegrees array
        cout << "Doing a round of scan on the cumDegs vector, size: " << cumDegSize << endl;

        uint *cCumDegs = new uint[cumDegSize];
        for (uint i = 0; i < cumDegSize; i++) {
            cCumDegs[i] = cumDegs[i];
        }

        if (cumDegSize < smemSize) {
            cout << "Resorting to a round of CPU scan" << endl;
            cpuScan(cumDegs, 0, cumDegSize);
            CHECK(cudaMemcpy(d_cumDegs, cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyHostToDevice));
        }
        else {
            uint *d_aux, *aux, *d_ogCumDegs;

            uint numSmemBlock = cumDegSize / smemSize;
            uint numBlock = (cumDegSize + blockDim - 1) / blockDim;
            uint gpuScanSize = numSmemBlock * smemSize;
            uint residualSize = cumDegSize - gpuScanSize;

            aux = (uint *) malloc((numSmemBlock + 1) * sizeof(uint));

            CHECK(cudaMalloc((void **) &d_aux, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMalloc((void **) &d_ogCumDegs, cumDegSize * sizeof(uint)));

            CHECK(cudaMemcpy(d_ogCumDegs, cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyHostToDevice));

            CHECK(cudaMemset(d_aux, 0, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));

            printf("\n  block scan...\n");

            uint smem = smemSize * sizeof(uint);
            printf("\n  first prescan procedure on the Device: %d elements...\n", gpuScanSize);
            cudaEventRecord(start);
            prescan<<<  numSmemBlock, blockDim, smem >>>(d_cumDegs, d_ogCumDegs, d_aux, size, smemSize);
            printf("\n  second scan procedure on the Host: %d elements...\n", residualSize);
            cpuScan(cumDegs, gpuScanSize, cumDegSize);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            float milliseconds;
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime = milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the contents of the aux array into Host memory and perform another scan
            printf("\n  third scan procedure on the Host: %d elements...\n", numSmemBlock);
            CHECK(cudaMemcpy(aux, d_aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyDeviceToHost));
            cudaEventRecord(start);
            cpuScan(aux, 0, numSmemBlock + 1);
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the portions of the array computed on the Host to Device memory
            CHECK(cudaMemcpy(d_aux, aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyHostToDevice));
            CHECK(cudaMemcpy(&(d_cumDegs[gpuScanSize]), &(cumDegs[gpuScanSize]), residualSize * sizeof(uint), cudaMemcpyHostToDevice));

            printf("\n  final summation procedure...\n");
            cudaEventRecord(start);
            final_sum<<< numBlock, blockDim >>>(d_cumDegs, d_aux, cumDegSize);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n\n", milliseconds / 1000.0);

            printf("\nTotal elapsed time:   %.5f (sec)\n", spliTime);

            totalTime += spliTime;
            CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

            free(aux);
            CHECK(cudaFree(d_aux));
            CHECK(cudaFree(d_ogCumDegs));
        }

        if (DEBUGGING) {
            for (uint i = 1; i < cumDegSize; i++) {
                cCumDegs[i] += cCumDegs[i - 1];
            }

            for (uint i = 0; i < cumDegSize - 1; i++) {
                if (cCumDegs[i] != cumDegs[i + 1]) {
                    cout << "I due array sono diversi in posizione " << i << endl;
                    cout << cCumDegs[i] << "   " << cumDegs[i + 1];
                    return -1;
                }
            }
        }

        cout << "The contracted graph will contain " << cumDegs[cumDegSize - 1] << " edges" << endl;
        cout << "The old graph structure contained " << str->edgeSize << " edges\n\n" << endl;




        // Allocating space for the arrays in the newly contracted graph
        uint newEdgeSize = cumDegs[cumDegSize - 1];
        node *newNeighs = new node[newEdgeSize];
        uint *newWeights = new uint[newEdgeSize];

        uint *d_newNeighs, *d_newWeights;
        CHECK(cudaMalloc((void **)&d_newNeighs, newEdgeSize * sizeof(node)));
        CHECK(cudaMalloc((void **)&d_newWeights, newEdgeSize * sizeof(uint)));
        CHECK(cudaMemset(d_newNeighs, 0, newEdgeSize * sizeof(node)));
        CHECK(cudaMemset(d_newWeights, 0, newEdgeSize * sizeof(uint)));

        cout << "Launching kernel GRAPH CONSTRUCTION -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        graphContraction<<<gridDim, blockDim>>>(str, d_colors, d_flag, d_cumDegs, d_newNeighs, d_newWeights);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("The construction of the new neighbour and weight arrays took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;
        CHECK(cudaMemcpy(newNeighs, d_newNeighs, newEdgeSize * sizeof(node), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(newWeights, d_newWeights, newEdgeSize * sizeof(uint), cudaMemcpyDeviceToHost));

        if (DEBUGGING) {
            node *checkNewNeighs = new node[newEdgeSize];
            uint *checkNewWeights = new uint[newEdgeSize];
            // Copy the contents of cumDegs into a new array
            for (uint i = 0; i < cumDegSize; i++) {
                cCumDegs[i] = cumDegs[i];
            }

            cudaEventRecord(start);
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node superVertex = getRoot(i, flag, colors);

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        int weight = str->getWeight(i, j);
                        uint position = cCumDegs[superVertex];
                        checkNewNeighs[position] = getRoot(neigh, flag, colors);
                        checkNewWeights[position] = weight;
                        cCumDegs[superVertex]++;
                    }
                }
            }

            cout << "I due array sono uguali" << endl;
            delete[] cCumDegs;
        }




        // Reconstructing the graph
        graphPointer->copyConstructor(newNodeSize, newEdgeSize, newNeighs, newWeights, cumDegs);

        //graphPointer->print(true);

        printf("----------------------------------\n\n");
        /***********************************************/




        // Updating the iteration information
        totalTime += spliTime;
        iterations++;
        /*****************************************/


        // Cuda memory deallocation
        CHECK(cudaFree(d_candidates));
        CHECK(cudaFree(d_colors));
        CHECK(cudaFree(d_flag));
        CHECK(cudaFree(d_cumDegs));
        CHECK(cudaFree(d_newNeighs));
        CHECK(cudaFree(d_newWeights));
        /****************************/

        // Host memory deallocation
        delete[] candidates;
        delete[] colors;
        delete[] flag;
        delete[] cumDegs;
        delete[] newNeighs;
        delete[] newWeights;
        /******************/
    }

    printf("Total elapsed time: %.5f seconds\n\n", totalTime);
    printf("The calculation of the MST took %d iterations\n\n", iterations);
    printf("The total weight of the tree is %d\n", mstWeight);


    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));

    return 0;
}

In [67]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75 GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/GPU/mstGPU.cu -o mstGPU
!./mstGPU

ptxas warning : Stack size for entry function '_Z17colorationProcessP11GraphStructPjS1_' cannot be statically determined
Processing a graph of size: 20000 with 199990050 edges.

Launching kernel FIND CHEAPEST -- (1024, 1, 1) -- (20, 1, 1)
Finding the cheapest edge for every vertex took: 0.34496 seconds

Launching kernel MIRRORED EDGES REMOVAL -- (1024, 1, 1) -- (20, 1, 1)
Removing the mirrored edges required: 0.10074 seconds

The MST weight at the end of iteration 1 is: 80484
Launching kernel COLORATION PROCESS -- (1024, 1, 1) -- (20, 1, 1)
Removing the mirrored edges required: 0.00008 seconds

Doing a round of scan on the flag vector, size: 20000

  block scan...

  first prescan procedure on the Device: 18432 elements...

  second scan procedure on the Host: 1568 elements...
   elapsed time:   0.00006 (sec)

  third scan procedure on the Host: 9 elements...
   elapsed time:   0.00000 (sec)

  final summation procedure...
   elapsed time:   0.00003 (sec)


Total elapsed time:   0.0000

In [ ]:
# ncu profiling

!nvcc -arch=sm_75 GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/GPU/mstGPU.cu -o mstGPU
!ncu --kernel-id ::graphContraction: -c 3 ./mstGPU

ptxas warning : Stack size for entry function '_Z17colorationProcessP11GraphStructPjS1_' cannot be statically determined
==PROF== Connected to process 32208 (/content/mstGPU)
Processing a graph of size: 20000 with 199987170 edges.

Launching kernel FIND CHEAPEST -- (256, 1, 1) -- (79, 1, 1)
Finding the cheapest edge for every vertex took: 0.25828 seconds

Launching kernel MIRRORED EDGES REMOVAL -- (256, 1, 1) -- (79, 1, 1)
Removing the mirrored edges required: 0.07163 seconds

Launching kernel COLORATION PROCESS -- (256, 1, 1) -- (79, 1, 1)
Removing the mirrored edges required: 0.00052 seconds

Launching a round of CPU scan
Doing the prefix sum of the auxiliary flag array took: 0.00008 seconds

Launching kernel CUMULATED DEGREE UPDATE -- (256, 1, 1) -- (79, 1, 1)
Doing the computation of the cumulated degrees took: 0.00856 seconds

Launching a round of CPU scan
Doing the scan of the new cumDegs array took: 0.00003 seconds

==PROF== Profiling "graphContraction": 0%....50%....100% - 9 pa

# GPU efficient approach

In [ ]:
%%cuda_group_save --name "mstGPUE.cu" --group "GPU"

// Header file di C++
#include <iostream>
#include <random>
#include <vector>
#include <algorithm>

// Header file C
#include <time.h>
#include <limits.h>

// Custom files
#include "../../GPUcomputing/utils/graph/graph_d.h"
#include "../../GPUcomputing/utils/graph/graph.h"
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

/*****
* Device function that gets the degree of a certain node
* @param str - The structure of the graph
* @param i - The node we are interested in
*****/
__device__ node d_deg (GraphStruct *str, node i) {
    return str->cumDegs[i + 1] - str->cumDegs[i];
}

/*****
* Device function that gets the weight of a certain edge
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ int d_getWeight (GraphStruct *str, node i, uint offset) {
    return str->weights[str->cumDegs[i] + offset];
}

/*****
* Device function that gets the neighbour of a certain node
* @param str - The structure of the graph
* @param i - The source node of the edge
* @param offset - The offset of the destination node in the adjacency list of
*                 the source
*****/
__device__ node d_getNeigh (GraphStruct *str, node i, uint offset) {
    return str->neighs[str->cumDegs[i] + offset];
}

__device__ uint d_getRoot (uint i, uint *d_flag, uint *d_colors) {
    return max(0, d_flag[d_colors[i]]);
}

uint getRoot (uint i, uint *flag, uint *colors) {
    return max(0, flag[colors[i]]);
}


/*****
* Kernel that finds the cheapest edge in the adjacency list of every node
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void findCheapest (GraphStruct *str, uint *d_candidates) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    // Initialize the minimum value
    uint minimum = UINT_MAX;
    int minimumWeight = INT_MAX;

    // Find the cheapest edge in each adjacency list
    for (uint i = 0; i < d_deg(str, idx); i++) {
        int edgeWeight = d_getWeight(str, idx, i);
        if (edgeWeight < minimumWeight) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
        else if (edgeWeight == minimumWeight &&
                 d_getNeigh(str, idx, i) < d_getNeigh(str, idx, minimum)) {
            minimumWeight = edgeWeight;
            minimum = i;
        }
    }

    // Update the return vector
    d_candidates[idx] = minimum;
}


/*****
* Kernel that removes the mirrored edges from the graph. A mirrored edge is
* simply an edge pointing from the source to the destination and vice versa in
* an oriented graph, the removal logic is to cut the edge with the lowest source
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
*****/
__global__ void mirroredEdgesRemoval (GraphStruct *str, uint *d_candidates, int *d_weight) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint destinationOffset = d_candidates[idx];
    node destination = d_getNeigh(str, idx, destinationOffset);
    if (idx < destination) {
        uint sourceOffset = d_candidates[destination];
        node destinationNeigh = d_getNeigh(str, destination, sourceOffset);

        // The vertex cannot be a candidate anymore because it would create a cycle
        if (destinationNeigh == idx) {
            d_candidates[idx] = UINT_MAX;
        }
    }

    if (d_candidates[idx] != UINT_MAX) {
        atomicAdd(d_weight, d_getWeight(str, idx, d_candidates[idx]));
    }
}


/*****
* Helper device function that recursively colors the nodes of the graph
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree (edges saved as offsets in the CSR
*                       representation of the graph)
* @param i - The index of the node to be colored
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__device__ uint *d_recursiveColorationHelper (GraphStruct *str, uint *d_candidates, node i, uint *d_colors) {
    uint color = UINT_MAX;
    if (d_candidates[i] == UINT_MAX) {
        color = i;
    }
    else {
        node neigh = d_getNeigh(str, i, d_candidates[i]);
        color = d_recursiveColorationHelper(str, d_candidates, neigh, d_colors)[neigh];
    }

    if (color != UINT_MAX) {
        d_colors[i] = color;
    }
    return d_colors;
}


/*****
* Kernel that recognizes the connected components in the graph and colors them
* @param str - The structure of the graph
* @param d_candidates - The device-level array of candidates to become part of
*                       the spanning tree
* @param d_colors - The device-level array of colors assigned to each vertex
*****/
__global__ void colorationProcess(GraphStruct *str, uint *d_candidates, uint *d_colors) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    d_recursiveColorationHelper(str, d_candidates, idx, d_colors);
}


/**
* Kernel that computes the prefix sun of the auxiliary flag array, code taken
* from the lectures and rearranged to follow the logic required for the
* implementation of the algorithm
* @param str - The structure of the graph
* @param d_colors - The device-level array of colors assigned to each vertex
* @param d_flag - The device-level array of flag values for the prefix sum.
*                 d_flag[i] = 1 if and only if the vertex in position i has the
*                 same color as the index => I recognize the root of the
*                 connected component
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void blockScan(uint *size, uint *input, uint *auxiliarVector) {
   __shared__ uint smem[BLOCK_SIZE];
   uint tid = threadIdx.x;
   uint idx = tid + blockIdx.x * blockDim.x;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   // Load input into shared memory.
   smem[tid] = input[idx];
   __syncthreads();

   // do recursive sums
   for (uint d = 1; d < BLOCK_SIZE; d *= 2) {
      if (tid >= d)
         smem[tid] += smem[tid - d];
      __syncthreads();
   }
   input[idx] = smem[tid];
   if (idx == ((blockIdx.x + 1) * blockDim.x - 1))
      auxiliarVector[blockIdx.x] = input[idx];
}


/**
* Auxiliary kernel that computes the sum of the partial values calculated using
* the blockScan kernel
* str - The structure of the graph
* @param d_flag - The device-level array containing intermediate sums obtained
*                 with the previous kernel
* @param d_auxiliarVector - The device-level array containing the last value of
*                           the prefix sum calculation for every block
**/
__global__ void sumBlockScan(uint *size, uint *input, uint *auxiliarVector) {
   uint idx = threadIdx.x + blockIdx.x * blockDim.x;  // global index 0:n-1
   uint s = 0;

   // If the index is out of bounds returns immediately
    if (idx >= *size) {
        return;
    }

   if (blockIdx.x == 0) return;

   for (uint j = 0; j < blockIdx.x; j++)
      s += auxiliarVector[j];

   // add term to input
   input[idx] += s;
}

//*** SCAN FUNCTIONS ***//

__global__ void prescan(uint *g_odata, uint *g_idata, uint *aux, int n, int smemSize)
{
  extern __shared__ int temp[];// allocated on invocation
  int thid = threadIdx.x;
  int offset = 1;
  int idx = blockIdx.x * blockDim.x + thid;

  temp[2*thid] = g_idata[2*idx]; // load input into shared memory
  temp[2*thid+1] = g_idata[2*idx+1];

  for (int d = n>>1; d > 0; d >>= 1) // build sum in place up the tree
  {
    __syncthreads();
    if (thid < d)
    {
      int ai = offset*(2*thid+1)-1;
      int bi = offset*(2*thid+2)-1;
      if (bi < smemSize && ai < smemSize) {
        temp[bi] += temp[ai];
      }
    }
    offset *= 2;
  }

  if (thid == 0)
  {
    aux[blockIdx.x] = temp[smemSize - 1];
    temp[smemSize - 1] = 0;
  } // clear the last element

  for (int d = 1; d < n; d *= 2) // traverse down tree & build scan
  {
    __syncthreads();
    if (thid < d && offset > 0)
    {
      int ai = offset*(2*thid+1)-1;
      int bi = offset*(2*thid+2)-1;
      if (bi < smemSize && ai < smemSize) {
        int t = temp[ai];
        temp[ai] = temp[bi];
        temp[bi] += t;
      }
    }
    offset >>= 1;
  }


  __syncthreads();
  if (idx <= (n / 2) - 1) {
      g_odata[2*idx] = temp[2*thid]; // write results to device memory
      g_odata[2*idx+1] = temp[2*thid+1];
  }
}

void cpuScan(uint *array, int start, int end) {
    if (end - start <= 1) {
        return;
    }

    int temp = array[start + 1];
    array[start + 1] = array[start];
    array[start] = 0;

    for (uint i = start + 1; i < end - 1; i++) {
        int sum = array[i] + temp;
        temp = array[i + 1];
        array[i + 1] = sum;
    }
}


__global__ void final_sum(uint *g_odata, uint *aux, uint n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  if (blockIdx.x == 0 || 2 * idx >= n) {
      return;
  }

  //printf("%d: ls - %d  rs - %d  aux - %d\n", idx, g_odata[2 * idx], g_odata[2 * idx + 1], aux[blockIdx.x - 1]);

  if (2 * idx == n - 1) {
      g_odata[2 * idx] += aux[blockIdx.x];
      return;
  }
  g_odata[2 * idx] += aux[blockIdx.x];
  g_odata[2 * idx + 1] += aux[blockIdx.x];
}

//****************//


__global__ void cumulatedDegreeUpdate(GraphStruct *str, uint *d_cumDegs, uint *d_colors, uint *d_flag) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node svSuccessor = d_getRoot(idx, d_flag, d_colors);
    uint sum = 0;

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            sum++;
        }
    }

    atomicAdd(&(d_cumDegs[svSuccessor]), sum);
}


__global__ void graphContraction(GraphStruct *str, uint *d_colors, uint *d_flag,
                                 uint *d_cumDegs, node *d_newNeighs, uint *d_newWeights) {
    uint idx = blockIdx.x * blockDim.x + threadIdx.x;

    // If the index is out of bounds returns immediately
    if (idx >= str->nodeSize) {
        return;
    }

    uint color = d_colors[idx];
    node superVertex = d_getRoot(idx, d_flag, d_colors);

    for (uint i = 0; i < d_deg(str, idx); i++) {
        node neigh = d_getNeigh(str, idx, i);
        uint neighColor = d_colors[neigh];

        if (color != neighColor) {
            int weight = d_getWeight(str, idx, i);
            uint position = atomicAdd(&(d_cumDegs[superVertex]), 1);
            d_newNeighs[position] = d_getRoot(neigh, d_flag, d_colors);
            d_newWeights[position] = weight;
        }
    }
}


int main () {
    // Generation of a random graph
    std::random_device rd;
    std::default_random_engine eng(FIXED_SEED);
    uint maxWeight = MAX_WEIGHT;
    float prob = .5;
    bool GPUEnabled = 1;
    Graph *graphPointer;
    Graph graph(SIZE, GPUEnabled);
    graphPointer = &graph;
  	graphPointer->randGraph(prob, true, maxWeight, eng);
    /**************************************************/


    // Checking if the random graph is connected
    if (!graphPointer->isConnected()) {
        cout << "The graph is not connected" << endl;
        return -1;
    }
    /************/


    uint iterations = 0;


    // Configuration of the GPU kernel
    uint blockDim = BLOCK_SIZE;
    uint *candidates;
    /***************/


    // Events to measure time
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    float milliseconds;
    float spliTime = 0;
    float totalTime = 0;
    /******************/


    // Variables calculating the MST weight
    int mstWeight = 0;
    int *d_mstWeight;
    CHECK(cudaMalloc((void **)&d_mstWeight, sizeof(int)));
    CHECK(cudaMemcpy(d_mstWeight, &mstWeight, sizeof(int), cudaMemcpyHostToDevice));
    /******************************************************************************/


    // Main block of the algorithm
    while (graphPointer->getStruct()->nodeSize > 1) {
        // Initialization of the variables associated with the graph
        GraphStruct *str = graphPointer->getStruct();
        uint size = str->nodeSize;
        uint edgeSize = str->edgeSize;
        cout << "Processing a graph of size: " << size << " with " << edgeSize << " edges.\n\n";
        uint gridDim = (size + blockDim - 1) / blockDim;
        if (DEBUGGING && size < 15 && str->edgeSize < 100) {
            graphPointer->print(true);
            print_d<<<1, 1>>>(str, 1);
            CHECK(cudaDeviceSynchronize());
        }
        candidates = new uint[size];
        /******************************/

        // First setp of the algorithm
        uint *d_candidates;
        CHECK(cudaMalloc((void**)&d_candidates, (size) * sizeof(uint)));
        CHECK(cudaMemset(d_candidates, 0, (size) * sizeof(uint)));
        cout << "Launching kernel FIND CHEAPEST -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        findCheapest<<<gridDim, blockDim>>>(str, d_candidates);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Finding the cheapest edge for every vertex took: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex
        if (DEBUGGING && size < 15) {
            cout << "The cheapest edge for every vertex" << endl;
            CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> " << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
            }
            cout << "\n\n\n";
        }
        /*******************/



        // Second step of the algorithm
        cout << "Launching kernel MIRRORED EDGES REMOVAL -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        mirroredEdgesRemoval<<<gridDim, blockDim>>>(str, d_candidates, d_mstWeight);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(candidates, d_candidates, (size) * sizeof(uint), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(&mstWeight, d_mstWeight, sizeof(int), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;
        /********************/

        // ~Debugging~ print the cheapest edge for every vertex update
        if (DEBUGGING && size < 15) {
            cout << "Update of the cheapest edge for every vertex" << endl;
            for (uint i = 0; i < size; i++) {
                cout << "node (" << i << ") -> ";
                if (candidates[i] != UINT_MAX) {
                    cout << str->getNeigh(i, candidates[i]) << "("
                    << str->getWeight(i, candidates[i]) << ")" << endl;
                }
                else {
                    cout << "NULL" << endl;
                }
            }
            printf ("%d\n", mstWeight);
        }
        /*****************************/

        cout << "The MST weight at the end of iteration " << iterations + 1 << " is: " << mstWeight << endl;



        // Third step of the algorithm
        cout << "Launching kernel COLORATION PROCESS -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        // Initialize the color array
        uint *colors = new uint[size];
        uint *d_colors;
        CHECK(cudaMalloc((void**)&d_colors, size * sizeof(uint)));
        CHECK(cudaMemset(d_colors, UINT_MAX, size * sizeof(uint)));
        /**************************************************/

        cudaEventRecord(start);
        colorationProcess<<<gridDim, blockDim>>>(str, d_candidates, d_colors);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        CHECK(cudaMemcpy(colors, d_colors, size * sizeof(uint), cudaMemcpyDeviceToHost));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Removing the mirrored edges required: %.5f seconds\n\n", spliTime);
        totalTime += spliTime;

        // Print the coloring
        if (DEBUGGING) {
            uint *checkColoring = new uint[size];

            for (uint i = 0; i < size; i++) {
                checkColoring[i] = 0;
            }

            for (uint i = 0; i < size; i++) {
                checkColoring[colors[i]]++;
            }

            uint nonZeroColors = 0;
            for (uint i = 0; i < size; i++) {
                if (checkColoring[i] != 0) {
                    nonZeroColors++;
                }
            }

            cout << "There is a total of " << nonZeroColors << " colors" << endl;

            cout << "\n\n\n";
        }
        /*******************/

        /**
         * If the coloring coming out of the last kernel contains only one color
         * then it means that the edge added in the last step was the one needed
         * to merge the partial trees
         **/
        uint color = colors[0];
        bool uniqueColor = true;
        for (uint i = 1; i < size; i++) {
            if (colors[i] != color) {
                uniqueColor = false;
                break;
            }
        }
        if (uniqueColor) {
            cout << "THE CALCULATION OF THE MST IS COMPLETE\n";
            cout << "THE MST WEIGHT IS: " << mstWeight << endl;
            printf("Total elapsed time: %.5f seconds\n\n", totalTime);

            // Cuda memory deallocation
            CHECK(cudaEventDestroy(start));
            CHECK(cudaEventDestroy(stop));
            CHECK(cudaFree(d_candidates));
            CHECK(cudaFree(d_colors));

            // Host memory deallocation
            delete[] candidates;
            delete[] colors;

            return 0;
        }
        /***********/




        // Fourth step of the algorithm
        cout << "Doing a round of scan on the flag vector, size: " << size << endl;
        uint *flag = new uint[size];
        uint *cFlag = new uint[size];
        for (uint i = 0; i < size; i++) {
            flag[i] = (colors[i] == i) ? 1 : 0;
            cFlag[i] = flag[i];
        }
        uint *d_flag;
        uint smemSize = 2 * blockDim;

        CHECK(cudaMalloc((void**)&d_flag, (size) * sizeof(uint)));

        if (size < smemSize) {
            cout << "Resorting to a round of CPU scan" << endl;
            cpuScan(flag, 0, size);
            CHECK(cudaMemcpy(d_flag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));
        }
        else {
            uint *d_aux, *aux, *d_ogFlag;

            uint numSmemBlock = size / smemSize;
            uint numBlock = (size + blockDim - 1) / blockDim;
            uint gpuScanSize = numSmemBlock * smemSize;
            uint residualSize = size - gpuScanSize;

            aux = (uint *) malloc((numSmemBlock + 1) * sizeof(uint));

            CHECK(cudaMalloc((void **) &d_aux, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMalloc((void **) &d_ogFlag, size * sizeof(uint)));

            CHECK(cudaMemcpy(d_ogFlag, flag, (size) * sizeof(uint), cudaMemcpyHostToDevice));

            CHECK(cudaMemset(d_aux, 0, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMemset(d_flag, 0, (size) * sizeof(uint)));

            printf("\n  block scan...\n");

            uint smem = smemSize * sizeof(uint);
            printf("\n  first prescan procedure on the Device: %d elements...\n", gpuScanSize);
            cudaEventRecord(start);
            prescan<<<  numSmemBlock, blockDim, smem >>>(d_flag, d_ogFlag, d_aux, size, smemSize);
            printf("\n  second scan procedure on the Host: %d elements...\n", residualSize);
            cpuScan(flag, gpuScanSize, size);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            float milliseconds;
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime = milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the contents of the aux array into Host memory and perform another scan
            printf("\n  third scan procedure on the Host: %d elements...\n", numSmemBlock);
            CHECK(cudaMemcpy(aux, d_aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyDeviceToHost));
            cudaEventRecord(start);
            cpuScan(aux, 0, numSmemBlock + 1);
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the portions of the array computed on the Host to Device memory
            CHECK(cudaMemcpy(d_aux, aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyHostToDevice));
            CHECK(cudaMemcpy(&(d_flag[gpuScanSize]), &(flag[gpuScanSize]), residualSize * sizeof(uint), cudaMemcpyHostToDevice));

            printf("\n  final summation procedure...\n");
            cudaEventRecord(start);
            final_sum<<< numBlock, blockDim >>>(d_flag, d_aux, size);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n\n", milliseconds / 1000.0);

            printf("\nTotal elapsed time:   %.5f (sec)\n", spliTime);

            totalTime += spliTime;
            CHECK(cudaMemcpy(flag, d_flag, (size) * sizeof(uint), cudaMemcpyDeviceToHost));

            free(aux);
            CHECK(cudaFree(d_aux));
            CHECK(cudaFree(d_ogFlag));
        }

        if (DEBUGGING) {
            for (uint i = 1; i < size; i++) {
                cFlag[i] += cFlag[i - 1];
            }

            for (uint i = 0; i < size - 1; i++) {
                if (cFlag[i] != flag[i + 1]) {
                    cout << "I due array sono diversi in posizione " << i << endl;
                    return -1;
                }
            }
        }
        delete[] cFlag;
        cout << "The contracted graph will contain " << flag[size - 1] << " supervertices\n\n" << endl;





        // Fifth step of the algorithm

        // Allocating resources for the new cumulated degrees array
        uint newNodeSize = flag[size - 1];
        uint cumDegSize = newNodeSize + 1;
        uint *cumDegs = new uint[cumDegSize];
        uint *d_cumDegs;
        CHECK(cudaMalloc((void**)&d_cumDegs, (cumDegSize) * sizeof(uint)));
        CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));
        /***********************************/

        cout << "Launching kernel CUMULATED DEGREE UPDATE -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;

        cudaEventRecord(start);
        cumulatedDegreeUpdate<<<gridDim, blockDim>>>(str, d_cumDegs, d_colors, d_flag);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        spliTime = milliseconds / 1000.0;
        printf("Doing the computation of the cumulated degrees took: %.5f seconds\n\n", spliTime);
        CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

        // ~Debugging~ looking for errors in the cumulated degrees array
        if (DEBUGGING) {
            uint *checkDegs = new uint[cumDegSize];
            for (uint i = 0; i < cumDegSize; i++) {
                checkDegs[i] = 0;
            }
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node svSuccessor = getRoot(i, flag, colors);
                uint sum = 0;

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        sum++;
                    }
                }

                checkDegs[svSuccessor] += sum;
            }
            for (uint i = 0; i < cumDegSize; i++) {
                if (cumDegs[i] != checkDegs[i]) {
                    cout << i << ": cumDegs - " << cumDegs[i] << "\tcheckDegs - " << checkDegs[i] << endl;
                    return -1;
                }
            }
            cout << "The CPU check vector and the GPU computed one are the same\n\n" << endl;
            delete[] checkDegs;
        }
        /********************/




        // Perform another prefix sum on the cumDegrees array
        cout << "Doing a round of scan on the cumDegs vector, size: " << cumDegSize << endl;

        uint *cCumDegs = new uint[cumDegSize];
        for (uint i = 0; i < cumDegSize; i++) {
            cCumDegs[i] = cumDegs[i];
        }

        if (cumDegSize < smemSize) {
            cout << "Resorting to a round of CPU scan" << endl;
            cpuScan(cumDegs, 0, cumDegSize);
            CHECK(cudaMemcpy(d_cumDegs, cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyHostToDevice));
        }
        else {
            uint *d_aux, *aux, *d_ogCumDegs;

            uint numSmemBlock = cumDegSize / smemSize;
            uint numBlock = (cumDegSize + blockDim - 1) / blockDim;
            uint gpuScanSize = numSmemBlock * smemSize;
            uint residualSize = cumDegSize - gpuScanSize;

            aux = (uint *) malloc((numSmemBlock + 1) * sizeof(uint));

            CHECK(cudaMalloc((void **) &d_aux, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMalloc((void **) &d_ogCumDegs, cumDegSize * sizeof(uint)));

            CHECK(cudaMemcpy(d_ogCumDegs, cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyHostToDevice));

            CHECK(cudaMemset(d_aux, 0, (numSmemBlock + 1) * sizeof(uint)));
            CHECK(cudaMemset(d_cumDegs, 0, (cumDegSize) * sizeof(uint)));

            printf("\n  block scan...\n");

            uint smem = smemSize * sizeof(uint);
            printf("\n  first prescan procedure on the Device: %d elements...\n", gpuScanSize);
            cudaEventRecord(start);
            prescan<<<  numSmemBlock, blockDim, smem >>>(d_cumDegs, d_ogCumDegs, d_aux, size, smemSize);
            printf("\n  second scan procedure on the Host: %d elements...\n", residualSize);
            cpuScan(cumDegs, gpuScanSize, cumDegSize);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            float milliseconds;
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime = milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the contents of the aux array into Host memory and perform another scan
            printf("\n  third scan procedure on the Host: %d elements...\n", numSmemBlock);
            CHECK(cudaMemcpy(aux, d_aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyDeviceToHost));
            cudaEventRecord(start);
            cpuScan(aux, 0, numSmemBlock + 1);
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

            // Copy the portions of the array computed on the Host to Device memory
            CHECK(cudaMemcpy(d_aux, aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyHostToDevice));
            CHECK(cudaMemcpy(&(d_cumDegs[gpuScanSize]), &(cumDegs[gpuScanSize]), residualSize * sizeof(uint), cudaMemcpyHostToDevice));

            printf("\n  final summation procedure...\n");
            cudaEventRecord(start);
            final_sum<<< numBlock, blockDim >>>(d_cumDegs, d_aux, cumDegSize);
            CHECK(cudaDeviceSynchronize());
            CHECK(cudaEventRecord(stop));
            CHECK(cudaEventSynchronize(stop));
            CHECK(cudaGetLastError());
            cudaEventElapsedTime(&milliseconds, start, stop);
            spliTime += milliseconds / 1000.0;
            printf("   elapsed time:   %.5f (sec)\n\n", milliseconds / 1000.0);

            printf("\nTotal elapsed time:   %.5f (sec)\n", spliTime);

            totalTime += spliTime;
            CHECK(cudaMemcpy(cumDegs, d_cumDegs, (cumDegSize) * sizeof(uint), cudaMemcpyDeviceToHost));

            free(aux);
            CHECK(cudaFree(d_aux));
            CHECK(cudaFree(d_ogCumDegs));
        }

        if (DEBUGGING) {
            for (uint i = 1; i < cumDegSize; i++) {
                cCumDegs[i] += cCumDegs[i - 1];
            }

            for (uint i = 0; i < cumDegSize - 1; i++) {
                if (cCumDegs[i] != cumDegs[i + 1]) {
                    cout << "I due array sono diversi in posizione " << i << endl;
                    cout << cCumDegs[i] << "   " << cumDegs[i + 1];
                    return -1;
                }
            }
        }

        cout << "The contracted graph will contain " << cumDegs[cumDegSize - 1] << " edges" << endl;
        cout << "The old graph structure contained " << str->edgeSize << " edges\n\n" << endl;




        // Allocating space for the arrays in the newly contracted graph
        uint newEdgeSize = cumDegs[cumDegSize - 1];
        node *newNeighs = new node[newEdgeSize];
        uint *newWeights = new uint[newEdgeSize];

        uint *d_newNeighs, *d_newWeights;
        CHECK(cudaMalloc((void **)&d_newNeighs, newEdgeSize * sizeof(node)));
        CHECK(cudaMalloc((void **)&d_newWeights, newEdgeSize * sizeof(uint)));
        CHECK(cudaMemset(d_newNeighs, 0, newEdgeSize * sizeof(node)));
        CHECK(cudaMemset(d_newWeights, 0, newEdgeSize * sizeof(uint)));

        cout << "Launching kernel GRAPH CONSTRUCTION -- (" << blockDim << ", 1, 1) -- (" << gridDim << ", 1, 1)" << endl;
        cudaEventRecord(start);
        graphContraction<<<gridDim, blockDim>>>(str, d_colors, d_flag, d_cumDegs, d_newNeighs, d_newWeights);
        CHECK(cudaDeviceSynchronize());
        CHECK(cudaEventRecord(stop));
        CHECK(cudaEventSynchronize(stop));
        cudaEventElapsedTime(&milliseconds, start, stop);
        printf("The construction of the new neighbour and weight arrays took: %.5f seconds\n\n", milliseconds/1000);
        spliTime += milliseconds / 1000.0;
        CHECK(cudaMemcpy(newNeighs, d_newNeighs, newEdgeSize * sizeof(node), cudaMemcpyDeviceToHost));
        CHECK(cudaMemcpy(newWeights, d_newWeights, newEdgeSize * sizeof(uint), cudaMemcpyDeviceToHost));

        if (DEBUGGING) {
            node *checkNewNeighs = new node[newEdgeSize];
            uint *checkNewWeights = new uint[newEdgeSize];
            // Copy the contents of cumDegs into a new array
            for (uint i = 0; i < cumDegSize; i++) {
                cCumDegs[i] = cumDegs[i];
            }

            cudaEventRecord(start);
            for (uint i = 0; i < size; i++) {
                uint color = colors[i];
                node superVertex = getRoot(i, flag, colors);

                for (uint j = 0; j < str->deg(i); j++) {
                    node neigh = str->getNeigh(i, j);
                    uint neighColor = colors[neigh];

                    if (color != neighColor) {
                        int weight = str->getWeight(i, j);
                        uint position = cCumDegs[superVertex];
                        checkNewNeighs[position] = getRoot(neigh, flag, colors);
                        checkNewWeights[position] = weight;
                        cCumDegs[superVertex]++;
                    }
                }
            }

            cout << "I due array sono uguali" << endl;
            delete[] cCumDegs;
        }




        // Reconstructing the graph
        graphPointer->copyConstructor(newNodeSize, newEdgeSize, newNeighs, newWeights, cumDegs);

        //graphPointer->print(true);

        printf("----------------------------------\n\n");
        /***********************************************/




        // Updating the iteration information
        totalTime += spliTime;
        iterations++;
        /*****************************************/


        // Cuda memory deallocation
        CHECK(cudaFree(d_candidates));
        CHECK(cudaFree(d_colors));
        CHECK(cudaFree(d_flag));
        CHECK(cudaFree(d_cumDegs));
        CHECK(cudaFree(d_newNeighs));
        CHECK(cudaFree(d_newWeights));
        /****************************/

        // Host memory deallocation
        delete[] candidates;
        delete[] colors;
        delete[] flag;
        delete[] cumDegs;
        delete[] newNeighs;
        delete[] newWeights;
        /******************/
    }

    printf("Total elapsed time: %.5f seconds\n\n", totalTime);
    printf("The calculation of the MST took %d iterations\n\n", iterations);
    printf("The total weight of the tree is %d\n", mstWeight);


    CHECK(cudaEventDestroy(start));
    CHECK(cudaEventDestroy(stop));

    return 0;
}

In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_75 GPUcomputing/utils/graph/graph.cpp GPUcomputing/utils/graph/graph_d.cu src/GPU/mstGPU.cu -o mstGPU
!./mstGPU

# Kernel testing

In [ ]:
%%cuda_group_save --name "scanTesting.cu" --group "TESTING"

// Header file di C++
#include <iostream>

// Header file C
#include <time.h>
#include <cstdlib>
#include <ctime>

// Custom files
#include "../../GPUcomputing/utils/common.h"
#include "../COMMON/sharedMacros.h"

using namespace std;

__global__ void prescan(uint *g_odata, uint *g_idata, uint *aux, int n, int smemSize)
{
  extern __shared__ int temp[];// allocated on invocation
  int thid = threadIdx.x;
  int offset = 1;
  int idx = blockIdx.x * blockDim.x + thid;

  temp[2*thid] = g_idata[2*idx]; // load input into shared memory
  temp[2*thid+1] = g_idata[2*idx+1];

  for (int d = n>>1; d > 0; d >>= 1) // build sum in place up the tree
  {
    __syncthreads();
    if (thid < d)
    {
      int ai = offset*(2*thid+1)-1;
      int bi = offset*(2*thid+2)-1;
      if (bi < smemSize && ai < smemSize) {
        temp[bi] += temp[ai];
      }
    }
    offset *= 2;
  }

  if (thid == 0)
  {
    aux[blockIdx.x] = temp[smemSize - 1];
    temp[smemSize - 1] = 0;
  } // clear the last element

  for (int d = 1; d < n; d *= 2) // traverse down tree & build scan
  {
    __syncthreads();
    if (thid < d && offset > 0)
    {
      int ai = offset*(2*thid+1)-1;
      int bi = offset*(2*thid+2)-1;
      if (bi < smemSize && ai < smemSize) {
        int t = temp[ai];
        temp[ai] = temp[bi];
        temp[bi] += t;
      }
    }
    offset >>= 1;
  }


  __syncthreads();
  if (idx <= (n / 2) - 1) {
      g_odata[2*idx] = temp[2*thid]; // write results to device memory
      g_odata[2*idx+1] = temp[2*thid+1];
  }
}

void cpuScan(uint *array, int start, int end) {
    if (end - start <= 1) {
        return;
    }

    int temp = array[start + 1];
    array[start + 1] = array[start];
    array[start] = 0;

    for (uint i = start + 1; i < end - 1; i++) {
        int sum = array[i] + temp;
        temp = array[i + 1];
        array[i + 1] = sum;
    }
}


__global__ void final_sum(uint *g_odata, uint *aux, uint n)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  if (blockIdx.x == 0 || 2 * idx >= n) {
      return;
  }

  //printf("%d: ls - %d  rs - %d  aux - %d\n", idx, g_odata[2 * idx], g_odata[2 * idx + 1], aux[blockIdx.x - 1]);

  if (2 * idx == n - 1) {
      g_odata[2 * idx] += aux[blockIdx.x];
      return;
  }
  g_odata[2 * idx] += aux[blockIdx.x];
  g_odata[2 * idx + 1] += aux[blockIdx.x];
}


/*
 * MAIN: test on parallel reduction
 */
int main(void) {
  uint *cpuArray, *gpuArray, *d_gpuArray, *gpuOutput, *d_gpuOutput, *aux, *d_aux;
  uint blockSize = 1024;
  uint smemSize = 2 * blockSize;
  uint n = 987423564;

  uint numSmemBlock = n / smemSize;
  uint numBlock = (n + blockSize - 1) / blockSize;
  uint gpuScanSize = numSmemBlock * smemSize;
  uint residualSize = n - gpuScanSize;

  // Memory allocation for the Host side
  cpuArray = (uint *) malloc(n * sizeof(uint));
  gpuArray = (uint *) malloc(n * sizeof(uint));
  gpuOutput = (uint *) malloc(n * sizeof(uint));
  aux = (uint *) malloc((numSmemBlock + 1) * sizeof(uint));

  // Memory allocation for the Device side
  CHECK(cudaMalloc((void **) &d_gpuArray, gpuScanSize * sizeof(uint)));
  CHECK(cudaMalloc((void **) &d_gpuOutput, n * sizeof(uint)));
  CHECK(cudaMalloc((void **) &d_aux, (numSmemBlock + 1) * sizeof(uint)));

  printf("\n****  test on parallel scan  ****\n");
	printf("  Vector length: %d\n", n);

  cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

  // Generate the original array
  srand(0);
	for (uint i = 0; i < n; i++){
     cpuArray[i] = 1;
     gpuArray[i] = cpuArray[i];
  }

	printf("\n  CPU procedure...\n");
	double go = seconds();
  for (uint i = 1; i < n; i++) {
      cpuArray[i] += cpuArray[i - 1];
  }
	double CPUtime = seconds() - go;
	printf("    Elapsed time: %f (sec) \n", CPUtime);

  // Copy the contents of gpuArray in the Device memory
  CHECK(cudaMemcpy(d_gpuArray, gpuArray, gpuScanSize * sizeof(uint), cudaMemcpyHostToDevice));
  CHECK(cudaMemset(d_aux, 0, (numSmemBlock + 1) * sizeof(uint)));

  printf("\n  block scan...\n");

  uint smem = smemSize * sizeof(uint);
  printf("\n  first prescan procedure on the Device: %d elements...\n", gpuScanSize);
  cudaEventRecord(start);
  prescan<<<  numSmemBlock, blockSize, smem >>>(d_gpuOutput, d_gpuArray, d_aux, n, smemSize);
  printf("\n  second scan procedure on the Host: %d elements...\n", residualSize);
  cpuScan(gpuArray, gpuScanSize, n);
  CHECK(cudaDeviceSynchronize());
	CHECK(cudaEventRecord(stop));
	CHECK(cudaEventSynchronize(stop));
	CHECK(cudaGetLastError());
  float milliseconds;
	cudaEventElapsedTime(&milliseconds, start, stop);
	double GPUtime = milliseconds / 1000.0;
	printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

  // Copy the contents of the aux array into Host memory and perform another scan
  printf("\n  third scan procedure on the Host: %d elements...\n", numSmemBlock);
  CHECK(cudaMemcpy(aux, d_aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyDeviceToHost));
  cudaEventRecord(start);
  cpuScan(aux, 0, numSmemBlock + 1);
  CHECK(cudaEventRecord(stop));
  CHECK(cudaEventSynchronize(stop));
  CHECK(cudaGetLastError());
	cudaEventElapsedTime(&milliseconds, start, stop);
	GPUtime += milliseconds / 1000.0;
	printf("   elapsed time:   %.5f (sec)\n", milliseconds / 1000.0);

  // Copy the portions of the array computed on the Host to Device memory
  CHECK(cudaMemcpy(d_aux, aux, (numSmemBlock + 1) * sizeof(uint), cudaMemcpyHostToDevice));
  CHECK(cudaMemcpy(&(d_gpuOutput[gpuScanSize]), &(gpuArray[gpuScanSize]), residualSize * sizeof(uint), cudaMemcpyHostToDevice));

  printf("\n  final summation procedure...\n");
  cudaEventRecord(start);
  final_sum<<< numBlock, blockSize >>>(d_gpuOutput, d_aux, n);
  CHECK(cudaDeviceSynchronize());
	CHECK(cudaEventRecord(stop));
	CHECK(cudaEventSynchronize(stop));
	CHECK(cudaGetLastError());
	cudaEventElapsedTime(&milliseconds, start, stop);
	GPUtime += milliseconds / 1000.0;
	printf("   elapsed time:   %.5f (sec)\n\n", milliseconds / 1000.0);

 	printf("\nTotal elapsed time:   %.5f (sec)\n", GPUtime);

	double speedup = CPUtime/GPUtime;
	printf("    Speedup %.1f\n", speedup);

  CHECK(cudaMemcpy(gpuOutput, d_gpuOutput, n * sizeof(uint), cudaMemcpyDeviceToHost));


  for (uint i = 0; i < n - 1; i++) {
      if (gpuOutput[i + 1] != cpuArray[i]) {
          printf("%d: %d\t%d\n", i - 1, gpuOutput[i - 1], cpuArray[i - 1]);
          printf("%d: %d\t%d\n", i, gpuOutput[i], cpuArray[i]);
          printf("%d: %d\t%d\n", i + 1, gpuOutput[i + 1], cpuArray[i + 1]);
          return -1;
      }
  }

  printf("%d - %d\n", gpuOutput[n - 1], cpuArray[n - 2]);
  printf("È andato tutto bene\n");

  // Host memory deallocation
  free(cpuArray);
  free(gpuArray);
  free(gpuOutput);
  free(aux);

  // Device memory deallocation
  CHECK(cudaFree(d_gpuArray));
  CHECK(cudaFree(d_gpuOutput));
  CHECK(cudaFree(d_aux));

	return 0;
}

In [ ]:
!nvcc -arch=sm_75 src/TESTING/scanTesting.cu -o scanTesting
!./scanTesting


****  test on parallel scan  ****
  Vector length: 987423564

  CPU procedure...
    Elapsed time: 2.873964 (sec) 

  block scan...

  first prescan procedure on the Device: 987422720 elements...

  second scan procedure on the Host: 844 elements...
   elapsed time:   0.31906 (sec)

  third scan procedure on the Host: 482140 elements...
   elapsed time:   0.00198 (sec)

  final summation procedure...
   elapsed time:   0.03480 (sec)


Total elapsed time:   0.35583 (sec)
    Speedup 8.1
987423563 - 987423563
È andato tutto bene


# Python Analysis

In [ ]:
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

resultsCPU = "/content/testing/primTest.txt"
resultsGPU = "/content/testing/gpuTest.txt"

pointerCPU = open(resultsCPU, "r")

sizes = []
times = []

bar_width = 0.35
bar_spacing = 0.5

for line in pointerCPU:
    splitLine = re.split(",", line)
    size = splitLine[0]
    time = float(splitLine[2])
    sizes.append(size)
    times.append(time)

logTimes = np.log2(times)
indices = np.arange(len(sizes)) * (1 + bar_spacing)

plt.bar(indices, logTimes, width=bar_width, edgecolor="white", linewidth=1,
        label="CPU")

pointerGPU = open(resultsGPU, "r")

sizes = []
times = []

for line in pointerGPU:
    splitLine = re.split(",", line)
    size = splitLine[0]
    time = float(splitLine[2])
    sizes.append(size)
    times.append(time)

logTimes = np.log2(times)
indices = np.arange(len(sizes)) * (1 + bar_spacing) + bar_width


plt.bar(indices, logTimes, width=bar_width, edgecolor="white", color="green",
        linewidth=1, label="GPU")
plt.xticks(indices - (bar_width / 2), sizes)

plt.legend()
plt.show()

close(pointerCPU)
close(pointerGPU)

FileNotFoundError: [Errno 2] No such file or directory: '/content/testing/primTest.txt'